<h1> Goal </h1>
We are going to train a Poker AI in Heads Up no limit holdem poker with a deep q learning algorithm. The algorithm is taken from ... with only very minor adjustments. I am very grateful to the author for sharing his code freely.

<h1> Creating the poker environment </h1>
Let's first load the libraries for the environment

In [12]:
import random
#import numpy as np
import itertools
agent="test"
print(agent)

values = list(range(2,15))
suits = [0, 1, 2, 3]
deck = []
for value in values:
    for suit in suits:
        deck.append((value, suit))

def deal_preflop(deck,n_players):
    hands = []                
    for i in range(n_players):
        hand = [deck[i], deck[i+n_players]]
        hand.sort()
        hands.append(tuple(hand))
    return hands

def deal_flop(deck,n_players):
    return (deck[n_players*2+1:n_players*2+4])

def deal_turn(deck,n_players):
    return deck[n_players*2+5]

def deal_river(deck,n_players):
    return deck[n_players*2+7]


#a function which gives the strength of a hand.

def check_straightflush(hand):
    hand = hand[:]
    hand.sort(reverse = True)
    for start in range(0,3):
        left = 6 - start

        inarow = 1
        current = start
        nxt = start +1
        while inarow + left > 4 and inarow <5:
            if hand[nxt][0] == hand[current][0] or (hand[nxt][0]+1 == hand[current][0] and hand[current][1] != hand[nxt][1]): #if nxt hand is same value or next in line, but not same suit:

                nxt += 1
                left -= 1

            elif hand[current][0] != hand[nxt][0]+1:
                break
            elif hand[current][1] != hand[nxt][1]:
                break

            else:
                inarow += 1
                left -= 1
                current = nxt
                nxt = current + 1

        if inarow >= 5:

            return True, hand[start][0]


    if inarow <5:
        straightflush = False
        #check if Ace to 5 straightflush is in hand:

        for suit in [0,1, 2, 3]:
            if (14, suit) in hand and (2,suit) in hand and (3, suit) in hand and (4, suit) in hand and (5, suit) in hand:

                straightflush = True
                return True, 5

        if not straightflush:

            return False


def check_quads(hand):
    handfreq = {}
    for card in hand:
        if card[0] not in handfreq:
            handfreq[card[0]]=1
        else:
            handfreq[card[0]]+= 1

    for hand in handfreq:
        if handfreq[hand]==4:
            quad = hand
            #höchste Beikarte ermitteln
            del(handfreq[hand])
            kicker = 2
            for card in handfreq.keys():
                if card > kicker:
                    kicker = card
            return True, [quad, kicker]

    return False


def check_fullhouse(hand):
    handfreq = {}
    for card in hand:
        if card[0] not in handfreq:
            handfreq[card[0]]=1
        else:
            handfreq[card[0]]+= 1
    if 3 in handfreq.values():
            trips = 2
            for hand in handfreq.keys():
                if handfreq[hand]==3 and hand >trips:
                    trips = hand
            del(handfreq[trips])
            pair = 1
            for hand in handfreq.keys():
                if handfreq[hand] in (2,3) and hand >pair:
                    pair = hand
            if pair >1:
                return True, [trips, pair]
    return False

def check_flush(hand):
    #count suits
    spades = []
    hearts = []
    clubs = []
    diamonds = []
    for card in hand:
        if card[1] ==0:
            spades.append(card[0])
        elif card[1] == 1:
            hearts.append(card[0])
        elif card[1] == 2:
            clubs.append(card[0])
        else: 
            diamonds.append(card[0])


    if len(spades) >=5:
        suit = spades
    elif len(hearts) >= 5:
        suit = hearts
    elif len(clubs) >= 5:
        suit = clubs
    elif len(diamonds)>= 5:
        suit = diamonds
    else:
        return False

    suit.sort(reverse=True)
    return True, suit[0:5]


def check_straight(hand):
    hand = hand[:]
    hand.sort(reverse = True)
    for start in range(0,3):
        left = 6 - start
        inarow = 1
        current = start
        nxt = start +1
        while inarow + left > 4 and inarow <5:

            if hand[nxt][0] == hand[current][0]:

                nxt += 1
                left -= 1

            elif hand[current][0] != hand[nxt][0]+1:
                break

            else:
                inarow += 1
                current = nxt
                nxt = current + 1
                left -=1
        if inarow >= 5:
            return True, hand[start][0]

    if inarow <5:
        straight = False
        #check if Ace to 5 straight is in hand:
        values = []
        for card in hand:
            values.append(card[0])
            if 14 in values and 2 in values and 3 in values and 4 in values and 5 in values:

                straight = True
                return True, 5

        if not straight:
            return False

def check_trips(hand):
    handfreq = {}
    for card in hand:
        if card[0] not in handfreq:
            handfreq[card[0]]=1
        else:
            handfreq[card[0]]+= 1
    for value in handfreq:
        if handfreq[value]==3:
            trips = value
            del(handfreq[value])
            cards=[]
            cards.append(trips)
            kickers = list(handfreq.keys())
            kickers.sort(reverse=True)
            cards.extend(kickers[0:2])
            return True, cards
    return False

def check_twopair(hand):
    handfreq = {}
    for card in hand:
        if card[0] not in handfreq:
            handfreq[card[0]]=1
        else:
            handfreq[card[0]]+= 1

    pairs = []
    highcard = 2
    for value in handfreq: #if freq == 2 append to pairs, else check for highcard

        if handfreq[value]==2:
            pairs.append(value)
        else:
            if value > highcard:
                highcard = value

    if len(pairs)>=2:
        pairs.sort(reverse=True)
        if len(pairs)>2 and pairs[2]>highcard:
            return True, [pairs[0], pairs[1], pairs[2]]
        return True, [pairs[0], pairs[1], highcard]
    else:
        return False

def check_pair(hand): #if True return list with pair-card and four strongest non-pair values
    hand = hand[:]
    hand.sort(reverse=True)
    cards = []
    for i in range(6):
        for j in range(i+1,7):

            if hand[i][0] == hand[j][0]:
                pair = hand[i][0]
                del(hand[i], hand[i])
                cards.append(pair)
                for k in range(3):
                    cards.append(hand[k][0])                                   
                return (True, cards)
    return False


def strength(hand): #Using the check hand functions above. Starting with the strongest hand. If hand is True, the function above returns True and the meaningful information about the hand. If not, it creates a
    #TypeError, because only False is retured, so nothing can be written into the output variable. We use this to profit from the try, except statements. This might look weired, but by that we only have to
    #run the functions one to test for the hand and create the output.

    try:
        a, output = check_straightflush(hand)
        return 8, output

    except TypeError:
        try:
            a, output = check_quads(hand)
            return 7, output
        except TypeError:
            try:
                a, output = check_fullhouse(hand)
                return 6, output
            except TypeError:
                try:
                    a, output = check_flush(hand)        
                    return 5, output
                except TypeError:
                    try:
                        a, output = check_straight(hand)
                        return 4, output
                    except TypeError:
                        try:
                            a, output = check_trips(hand)
                            return 3, output
                        except TypeError:
                            try:
                                a, output = check_twopair(hand)
                                return 2, output
                            except TypeError:
                                try:
                                    a, output = check_pair(hand)
                                    return  1, output
                                except TypeError:
                                    #if hand is not pair, it's highcard. We return strength of 0 and a sorted list of cardvalues
                                    highcards = []
                                    hand = hand[:]
                                    hand.sort(reverse = True)
                                    for i in range(5):
                                        highcards.append(hand[i][0])
                                    return 0, highcards

def compare(hands): #takes in a dictonary of hands with position as keys (sb = 0) and returns dictonary of positions with rank (0 is strongest)
    rankings = hands.copy()
    for position in hands:
        rankings[position] = (strength(hands[position]))
    #create a sorted list of tuples with positions and strengths
    sorted_hands = sorted(rankings.items(), key=lambda x: x[1], reverse=True)
    #create a dictonary of positions and ranking. Equal strengths should get same ranking.
    ranking_dict = {sorted_hands[0][0]:0}
    for i in range(1, len(sorted_hands)):
        if sorted_hands[i][1] == sorted_hands[i-1][1]:
            ranking_dict[sorted_hands[i][0]]=i-1
        else:
            ranking_dict[sorted_hands[i][0]]=i
    return ranking_dict

class HUPoker:   
    def __init__(self, stack_hero, stack_villain,random_stack = False, minstack = 5, maxstack = 200, n_run =20, hero=agent, villain = agent): #we are going to train the AI by playing against itself. Later we are going to implement
        #a user interface, to play against. 
        self.starting_hero = stack_hero
        self.starting_villain = stack_villain
        self.n_run = n_run #we want to reward plays by the aproximate ev, so here we define how many times to board is run after a showdown prior to the river.
        self.hero = hero #the agents that interact with the environment. 
        self.villain = villain
        self.random_stack = random_stack
        self.minstack, self.maxstack  = minstack, maxstack


    def reset(self): #shuffles the deck, seats the two players and returns position, stack sizes, hand (only suited or unsuited, because actual stuits are not important preflop) 
        if self.random_stack:
            self.starting_hero = np.random.randint(self.minstack,self.maxstack) 
            self.starting_villain = np.random.randint(self.minstack,self.maxstack)
        #keep track of action counts
        self.count_round = -1

        self.position = np.random.randint(2)
        if self.position == 0:
            self.stack_sb = self.starting_hero - 0.5 
            self.stack_bb = self.starting_villain - 1
        else:
            self.stack_bb = self.starting_hero - 1 
            self.stack_sb = self.starting_villain - 0.5
        self.bet_sb = 0.5 #keeps track of the bet chips by player in current street
        self.bet_bb = 1
        self.left_bet_round = [1,0] #which players are left to bet in this round
        self.pot = self.bet_sb + self.bet_bb    
        self.street = 0 #0: preflop, 1:flop, 2: turn, 3: river

        random.shuffle(deck)
        self.board_current = []

        self.holecards = deal_preflop(deck,2)[self.position]
        self.holecards_villain = deal_preflop(deck, 2)[0] if self.position == 1 else deal_preflop(deck, 2)[1]
        self.value_low = self.holecards[0][0]
        self.value_high = self.holecards[1][0]
        if self.holecards[0][1] == self.holecards[1][1]:
            self.suited =1
        else:
            self.suited = 0


        self.action_villain = -1 #meaningless value

        #initiate hand history to -1
        self.hh =[[],[],[],[]]
        for street in range(4):
            for i in range(20):
                self.hh[street].append(-1)

        self.suit_low = -1
        self.suit_high = -2
        self.flop1val = -1 #meaningless values
        self.flop1suit = -1 #meaningless values
        self.flop2val = -1 #meaningless values
        self.flop2suit = -1 #meaningless values
        self.flop3val = -1 #meaningless values
        self.flop3suit = -1 #meaningless values
        self.turnval = -1
        self.turnsuit = -1
        self.riverval = -1
        self.riversuit = -1
        done = False

        #constructing observation for villain to chose action on.
        if self.position == 1:
            position = 0
            value_low = self.holecards_villain[0][0]
            suit_low = self.holecards_villain[0][1]
            value_high = self.holecards_villain[1][0]
            suit_high = self.holecards_villain[1][1]
            suited = 1 if suit_low == suit_high else 0
            if self.street == 0:
                suit_high, suit_low = -1, -1

            observation_villain = self.create_observation_villain()
            self.action_villain = self.villain.choose_action(observation_villain)

            if self.action_villain == 0:
                self.stack_bb += 1.5
                bet = 0
                done = True

            elif self.action_villain == 1:
                bet = 0.5

            elif self.action_villain >= 2 and self.action_villain <6:#minraise - at least complete plus the last raised bit
                bet = 1.5

            elif self.action_villain == 6 or self.action_villain == 7: #2.5 total bet villain
                bet = 2.5*self.bet_bb - self.bet_sb

            elif self.action_villain == 8 or self.action_villain == 9: #3 total bet villain
                bet = 3*self.bet_bb - self.bet_sb

            elif self.action_villain == 10: #4 total bet villain
                bet = 4*self.bet_bb - self.bet_sb

            else: #allin
                bet = self.stack_sb

            self.bet_sb += bet
            self.stack_sb -= bet
            self.pot += bet
            self.left_bet_round = [1]
            self.hh[0][0] = bet
            self.count_round = 0
        self.create_observation()

        return self.observation, done


    def create_observation(self): #creates the observation array. A state in poker consists of all actions made in the entire hand. So we have to save all actions by hero and villain.
        #the best way to do so is to keep track of all bet sizes. We need enough empty slots to keep track of all bets. A check gets the value 0.
            self.observation = np.asarray([self.position, self.stack_sb, self.stack_bb, self.street, self.value_low, self.value_high, self.suited,  
                                       self.pot, self.bet_sb, self.bet_bb, self.suit_low, self.suit_high, 
                                        self.hh[0][0], self.hh[0][1], self.hh[0][2], self.hh[0][3],self.hh[0][4], self.hh[0][5], self.hh[0][6],self.hh[0][7], self.hh[0][8], self.hh[0][9],
                                        self.hh[0][10], self.hh[0][11], self.hh[0][12], self.hh[0][13],self.hh[0][14], self.hh[0][15], self.hh[0][16],self.hh[0][17], self.hh[0][18], self.hh[0][19], 
                                        self.hh[1][0], self.hh[1][1], self.hh[1][2], self.hh[1][3],self.hh[1][4], self.hh[1][5], self.hh[1][6],self.hh[1][7], self.hh[1][8], self.hh[1][9],
                                        self.hh[1][10], self.hh[1][11], self.hh[1][12], self.hh[1][13],self.hh[1][14], self.hh[1][15], self.hh[1][16],self.hh[1][17], self.hh[1][18], self.hh[1][19], 
                                        self.hh[2][0], self.hh[2][1], self.hh[2][2], self.hh[2][3],self.hh[2][4], self.hh[2][5], self.hh[2][6],self.hh[2][7], self.hh[2][8], self.hh[2][9],
                                        self.hh[2][10], self.hh[2][11], self.hh[2][12], self.hh[2][13],self.hh[2][14], self.hh[2][15], self.hh[2][16],self.hh[2][17], self.hh[2][18], self.hh[2][19],
                                        self.hh[3][0], self.hh[3][1], self.hh[3][2], self.hh[3][3],self.hh[3][4], self.hh[3][5], self.hh[3][6],self.hh[3][7], self.hh[3][8], self.hh[3][9],
                                        self.hh[3][10], self.hh[3][11], self.hh[3][12], self.hh[3][13],self.hh[3][14], self.hh[3][15], self.hh[3][16],self.hh[3][17], self.hh[3][18], self.hh[3][19],
                                       self.flop1val, self.flop1suit, self.flop2val, self.flop2suit, self.flop3val, self.flop3suit, 
                                           self.turnval, self.turnsuit, self.riverval, self.riversuit]).astype(np.float32)

    def create_observation_villain(self): #this is the last action by hero. If two betting rounds in a row, set to -1
        position = 0 if self.position == 1 else 1
        value_low = self.holecards_villain[0][0]
        suit_low = self.holecards_villain[0][1] if self.street != 0 else -1
        value_high = self.holecards_villain[1][0]
        suit_high = self.holecards_villain[1][1] if self.street != 0 else -1
        suited = 1 if suit_low == suit_high else 0


        return np.asarray([position, self.stack_sb, self.stack_bb, self.street, value_low, value_high, suited,  
                                   self.pot, self.bet_sb, self.bet_bb, suit_low, suit_high,
                                   self.hh[0][0], self.hh[0][1], self.hh[0][2], self.hh[0][3],self.hh[0][4], self.hh[0][5], self.hh[0][6],self.hh[0][7], self.hh[0][8], self.hh[0][9],
                                    self.hh[0][10], self.hh[0][11], self.hh[0][12], self.hh[0][13],self.hh[0][14], self.hh[0][15], self.hh[0][16],self.hh[0][17], self.hh[0][18], self.hh[0][19], 
                                    self.hh[1][0], self.hh[1][1], self.hh[1][2], self.hh[1][3],self.hh[1][4], self.hh[1][5], self.hh[1][6],self.hh[1][7], self.hh[1][8], self.hh[1][9],
                                    self.hh[1][10], self.hh[1][11], self.hh[1][12], self.hh[1][13],self.hh[1][14], self.hh[1][15], self.hh[1][16],self.hh[1][17], self.hh[1][18], self.hh[1][19], 
                                    self.hh[2][0], self.hh[2][1], self.hh[2][2], self.hh[2][3],self.hh[2][4], self.hh[2][5], self.hh[2][6],self.hh[2][7], self.hh[2][8], self.hh[2][9],
                                    self.hh[2][10], self.hh[2][11], self.hh[2][12], self.hh[2][13],self.hh[2][14], self.hh[2][15], self.hh[2][16],self.hh[2][17], self.hh[2][18], self.hh[2][19],
                                    self.hh[3][0], self.hh[3][1], self.hh[3][2], self.hh[3][3],self.hh[3][4], self.hh[3][5], self.hh[3][6],self.hh[3][7], self.hh[3][8], self.hh[3][9],
                                    self.hh[3][10], self.hh[3][11], self.hh[3][12], self.hh[3][13],self.hh[3][14], self.hh[3][15], self.hh[3][16],self.hh[3][17], self.hh[3][18], self.hh[3][19],
                                    self.flop1val, self.flop1suit, self.flop2val, self.flop2suit, self.flop3val, self.flop3suit, 
                                    self.turnval, self.turnsuit, self.riverval, self.riversuit]).astype(np.float32)

    def implement_action(self, action, player = 'hero'): #implements either heros or villains action to the point where either the next player is to act or the hand is finished
        bet = 0
        self.count_round += 1
        done =False
        next_to_act = self.left_bet_round.pop() #last in list acts first. BB acts last preflop, which we allready tool care of above. Otherwise BB acts first.
        #transform action into legal option
        if action == 0:
            if self.bet_sb == self.bet_bb: #player cannot fold, if he can check
                action = 1

        elif action > 1: #player cannot raise, if he has less than pot - self.bet
            if (player == 'hero' and self.position == 0 and self.bet_bb - self.bet_sb >= self.stack_sb) or (
                player == 'hero' and self.position == 1 and self.bet_sb - self.bet_bb >= self.stack_bb) or(
                player == 'villain' and self.position == 1 and self.bet_bb - self.bet_sb >= self.stack_sb) or (
                player == 'villain' and self.position == 0 and self.bet_sb - self.bet_bb >= self.stack_bb):
                action = 1


        if action == 0: #player folds. Action is already transformed into 1, if player is first in.
            if next_to_act == 0: #hero was sb
                self.stack_bb += self.pot

            else: #player is bb
                self.stack_sb += self.pot
            self.left_bet_round = []
            done = True #round is finished. self.reset should be performed outside this function
            self.hh[self.street][self.count_round] = bet
            done=True
        elif action == 1: #player calls/checks
            if next_to_act == 0: #player is sb
                if self.bet_sb == self.bet_bb: #bets are ballanced, so player can check.

                    pass               

                else: #bets are unbalanced, so player calls
                    left_to_bet = self.bet_bb - self.bet_sb
                    if left_to_bet < self.stack_sb: #hero doesn't have to go allin to call
                        bet = self.bet_bb-self.bet_sb
                        self.bet_sb = self.bet_bb
                        self.stack_sb -= left_to_bet
                        self.pot += left_to_bet

                    else: #player goes allin
                        self.pot += self.stack_sb
                        self.bet_sb = self.bet_sb + self.stack_sb
                        bet = self.stack_sb
                        self.stack_sb = 0
                        #give bb chips higher than sb_stack back
                        bet_difference = self.bet_bb - self.bet_sb
                        self.stack_bb += bet_difference
                        self.pot -= bet_difference
                        #self.hh[self.street][self.count_round] = bet
                        self.showdown()
                        done = True


            else: #player is bb
                if self.bet_sb == self.bet_bb: #bets are ballanced, so player can check.
                    pass


                else: #bets are unbalanced, so player calls
                    left_to_bet = self.bet_sb - self.bet_bb
                    if left_to_bet <= self.stack_bb: #hero doesn't have to go allin to call
                        bet = self.bet_sb -self.bet_bb
                        self.bet_bb = self.bet_sb
                        self.stack_bb -= left_to_bet
                        self.pot += left_to_bet
                        self.left_bet_round = []


                    else: #plyer goes allin 
                        self.pot += self.stack_bb
                        self.bet_bb = self.bet_bb + self.stack_bb
                        bet = self.stack_bb
                        self.stack_bb = 0
                        #give bb chips higher than bb_stack back
                        bet_difference = self.bet_sb - self.bet_bb
                        self.stack_sb += bet_difference
                        self.pot -= bet_difference


                        #self.hh[self.street][self.count_round] = bet
                        self.showdown()
                        done = True


        else: #player raises/bets From here on we have to change the code.
            # if unbet the bet variable indicates how much the player will increase his current betsize.
            if next_to_act == 0 and self.bet_bb == 0 or next_to_act == 1 and self.bet_sb == 0:
                if action == 2:
                    bet = 1
                elif action == 3:
                    bet = self.pot//10.0
                    if bet < 1: #bet has to be at least 1BB
                        bet = 1
                elif action == 4:
                    bet = self.pot//4.0
                    if bet < 1: #bet has to be at least 1BB
                        bet = 1
                elif action ==5:
                    bet = self.pot//3.0
                    if bet < 1: #bet has to be at least 1BB
                        bet = 1

                elif action == 6:
                    bet = self.pot//2.0
                elif action ==7:
                    bet = 3*self.pot//4.0
                elif action == 8:
                    bet = self.pot
                elif action == 9:
                    bet = self.pot*13//10.0
                elif action == 10:
                    bet = self.pot*2

                else:
                    bet = self.stack_sb if self.position == 0 else self.stack_bb

            #if pot is bet:
            else:

                if action == 2 or action == 3:#minraise - at least complete plus the last raised bit
                    if next_to_act == 0:
                        #preflop first in
                        if self.street == 0 and self.bet_sb == 0.5:
                            bet = 1.5
                        else:
                            bet = 2*(self.bet_bb - self.bet_sb)
                    else:
                        bet = 2*(self.bet_sb - self.bet_bb)
                        if bet < 1:
                            bet = 1#at least a bet of 1
                elif action == 4 or action == 5: # 2 total bet villain
                    if next_to_act == 0:
                        bet = 2*self.bet_bb - self.bet_sb
                    else:
                        bet = 2*self.bet_sb - self.bet_bb
                elif action == 6 or action == 7: #2.5 total bet villain
                    if next_to_act == 0:
                        bet = 2.5*self.bet_bb - self.bet_sb
                    else:
                        bet = 2.5*self.bet_sb - self.bet_bb

                elif action == 8 or action == 9: #3 total bet villain
                    if next_to_act == 0:
                        bet = 3*self.bet_bb - self.bet_sb
                    else:
                        bet = 3*self.bet_sb - self.bet_bb
                elif action == 10: #4 total bet villain
                    if next_to_act == 0:
                        bet = 4*self.bet_bb - self.bet_sb
                    else:
                        bet = 4*self.bet_sb - self.bet_bb

                else: #allin
                    bet = self.stack_sb if next_to_act == 0 else self.stack_bb

            if (bet >= self.stack_sb and next_to_act == 0) or (bet >= self.stack_bb and next_to_act == 1): #allin if not enough chips left.
                bet = self.stack_sb if next_to_act == 0 else self.stack_bb



            if next_to_act == 0: #player is sb
                self.bet_sb += bet
                self.pot += bet
                self.stack_sb -= bet
                self.left_bet_round = [1] if self.stack_bb != 0 else [] #on an agressive move, his opponent is next to act. If player went all in, no further action.

            else:
                self.bet_bb += bet
                self.pot += bet
                self.stack_bb -= bet
                self.left_bet_round = [0] if self.stack_sb != 0 else []


        if len(self.left_bet_round) == 0:
            if not done:
                self.bet_sb = 0 #put bet in round back to 0
                self.bet_bb = 0

                if self.street == 3 or self.stack_sb == 0 or self.stack_bb == 0:
                    self.hh[self.street][self.count_round] = bet
                    self.showdown()
                    done = True
                    return done

                if self.street == 0:
                    self.hh[self.street][self.count_round] = bet
                    self.street = 1
                    self.count_round =-1

                    self.suit_low = self.holecards[0][1]
                    self.suit_high = self.holecards[1][1]
                    flop = deal_flop(deck,2)
                    self.board_current = flop
                    self.flop1val = flop[0][0]
                    self.flop1suit = flop[0][1]
                    self.flop2val = flop[1][0]
                    self.flop2suit = flop[1][1]
                    self.flop3val = flop[2][0]
                    self.flop3suit = flop[2][1]

                    self.left_bet_round = [0,1]

                    self.create_observation()
                    return done

                elif self.street ==1:
                    self.hh[self.street][self.count_round] = bet
                    self.street = 2
                    self.count_round = -1
                    turn = deal_turn(deck,2)
                    self.board_current.append(turn)

                    self.turnval = turn[0]
                    self.turnsuit = turn[1]

                    self.left_bet_round = [0,1]


                    self.create_observation()
                    return done

                else:
                    self.hh[self.street][self.count_round] = bet
                    self.street = 3
                    self.count_round = -1
                    river = deal_river(deck,2)
                    self.board_current.append(river)
                    self.riverval = river[0]
                    self.riversuit = river[1]

                    self.left_bet_round = [0,1]


                    self.create_observation()
                    return done

        self.hh[self.street][self.count_round] = bet
        return done       


    def step(self, action): #implements hero's action all up to the point where either the hand is over, or the next action is asked
        done = self.implement_action(action, 'hero')
        self.action_villain = -1 #if villain has no action after hero, set back to no action. Meaning that hero bets two times in a row
        if not done and self.left_bet_round != []:
            if self.left_bet_round[-1] != self.position: #villain is next to act.
                #constructing observation for villain to chose action on.            
                observation_villain = self.create_observation_villain()
                self.action_villain = self.villain.choose_action(observation_villain)  
                done = self.implement_action(self.action_villain,'villain')

                   #check if another villains action must be implemented.
                if not done and self.left_bet_round != []: #player left in round.
                    if self.left_bet_round[-1] != self.position: #next to act is not hero.
                    #check if we have to implement another villains action. 
                        observation_villain = self.create_observation_villain()
                        self.action_villain = self.villain.choose_action(observation_villain)            
                        done = self.implement_action(self.action_villain,'villain')
        self.create_observation()
        return done







    def showdown(self):

        if self.street == 0:
            cards_left = 5 #how many cards left to deal
        else:
            cards_left = 3 - self.street

        share_pot_sb, share_pot_bb = 0, 0
        for i in range(self.n_run):

            board = self.board_current[:]


            board_remaining = random.sample(deck[8:],cards_left) #we don't have to take into accounts the cards already dealt, because we start with the river card anyway.
            board.extend(board_remaining)
            #self.board_current.extend(board_remaining)


            hand_hero_total = list(self.holecards) + board
            hand_villain_total=list(self.holecards_villain) + board

            comparison = compare({0:hand_hero_total, 1:hand_villain_total})
            if comparison[0] < comparison[1]:

                if self.position == 0:                    
                    share_pot_sb += self.pot
                else:
                    share_pot_bb += self.pot
            elif comparison[0] == comparison[1]:

                share_pot_sb += self.pot/2
                share_pot_bb += self.pot/2

            else:

                if self.position == 0:
                    share_pot_bb += self.pot
                else:
                    share_pot_sb += self.pot


        self.stack_sb += share_pot_sb/self.n_run

        self.stack_bb += share_pot_bb/self.n_run

        self.create_observation()
        done = True
        #return done

def play_against(n_games = 100000,hero = agent, villain=agent, stack_hero=100, stack_villain=100):
    hero = hero
    villain = villain
    env = HUPoker(stack_hero,stack_villain, hero=hero, villain = villain)
    total_score = 0

    for i in range(n_games):
        observation, done = env.reset()
        reward = 0

        while not done:
            action = hero.choose_action(observation)
            done = env.step(action)
            observation = env.observation



        reward = env.stack_sb - env.starting_hero if env.position == 0 else env.stack_bb - env.starting_hero 

        total_score += reward
        avg_score = total_score/(i+0.00001)
        print('episode: ', i, 'reward %.2f' % reward,
                        'average_score %.2f' % avg_score,
                        'total score %.2f' % total_score)
    return avg_score, env.observation

class PokerInterface():
    def __init__(self, hero=agent, villain = agent, stack_hero=100, stack_villain=100):
        self.hero = hero
        self.villain = villain
        self.poker = HUPoker(stack_hero,stack_villain, hero= self.hero, villain = self.villain)


    def simulate(self):
        poker = self.poker
        observation = poker.reset()
        hand = poker.holecards
        pos_player = ['hero','villain'] if poker.position == 0 else ['villain','hero']

        value_dict = {10: 'T', 11: 'J', 12: 'Q', 13: 'K', 14: 'A'}
        for i in range (2,10):
            value_dict[i] = str(i)
        suit_dict = {0 : '\u2660', 1:'\u2665', 2: '\u2663', 3: '\u2666'} #creating suits from unicode
        dict_action = {0: 'folds', 1: 'checks/calls', 2: 'minbets/raises', 3: 'bets/raises', 4: 'bets/raises', 5: 'bets/raises', 6: 'bets/raises', 7: 'bets/raises',
                       8: 'bets/raises', 9: 'bets/raises', 10: 'bets/raises', 11: 'goes all in',}

        card1 = value_dict[hand[0][0]]+suit_dict[hand[0][1]]
        card2 = value_dict[hand[1][0]]+suit_dict[hand[1][1]]
        print(pos_player[0], "is the button")            
        print("Stack Hero ", poker.starting_hero)
        print("Stack Villain", poker.starting_villain)
        print(pos_player[0], 'posts small blind 0.5')
        print(pos_player[1], 'posts big blind 1')
        print("*** HOLE CARDS ***")
        print("Dealt to Hero ", card1, card2)

        done = False
        if poker.position == 1:
            action = poker.action_villain
            print('Villain', dict_action[action])
            if action == 0:
                done = True
            poker.left_bet_round = [1]

        next_to_act = 'hero'            


        while not done:
            if poker.bet_sb == poker.bet_bb:
                next_to_act = 'hero' if poker.position == 1 else 'villain'
            street = poker.street
            if street != 0:
                print("Stack Hero ", poker.stack_sb if poker.position == 0 else poker.stack_bb)
                print("Stack Villain ", poker.stack_sb if poker.position == 1 else poker.stack_bb)
                board = ""
                for i in range(len(poker.board_current)):
                            board += value_dict[poker.board_current[i][0]]+suit_dict[poker.board_current[i][1]]
                if street == 1:
                    print("*** FLOP ***", board)

                elif street == 2:
                    print("*** TURN ***", board[:-2], " ", board[-2:])

                else:
                    print("*** RIVER ***", board[:-2], " ", board[-2:])

            while street == poker.street and not done:
                if next_to_act == 'hero':
                    poker.create_observation()
                    observation = poker.observation
                    action = self.hero.choose_action(observation)

                else:

                    observation_villain = poker.create_observation_villain()
                    action = self.villain.choose_action(observation_villain)


                done = poker.implement_action(action, player = next_to_act)
                print("done =", done)
                print(next_to_act, action, dict_action[action])
                next_to_act = 'villain' if next_to_act == 'hero' else 'hero'

        if action != 0: #if noone folded
            print("*** SHOW DOWN ***")
            card1_villain = value_dict[poker.holecards_villain[0][0]]+suit_dict[poker.holecards_villain[0][1]]
            card2_villain = value_dict[poker.holecards_villain[1][0]]+suit_dict[poker.holecards_villain[1][1]]
            if poker.position == 0:
                print(pos_player[0], "shows", card1, card2 )
                print(pos_player[1], "shows", card1_villain, card2_villain)
            else:
                print(pos_player[0], "shows", card1_villain, card2_villain)
                print(pos_player[1], "shows", card1, card2 )

            board = ""
            for i in range(len(poker.board_current)):
                        board += value_dict[poker.board_current[i][0]]+suit_dict[poker.board_current[i][1]]
            if street < 1:
                print("*** FLOP ***", board[:6])

            if street < 2:
                print("*** TURN ***", board[:6], " ", board[6:-2])

            if street < 3:
                print("*** RIVER ***", board[:-2], " ", board[-2:])

        if (poker.stack_sb > poker.starting_hero and poker.position == 0) or poker.stack_bb > poker.starting_hero and poker.position == 1:
            print("Hero wins", poker.pot)
        elif poker.stack_sb == poker.starting_hero or poker.stack_bb == poker.starting_hero:
            print("Split pot. Bot players receive", poker.pot/2)
        else:
            print("Villain wins", poker.pot)

        print("*** SUMMARY ***")
        print("Difference stack", pos_player[0], poker.stack_sb - poker.starting_hero if poker.position == 0 else poker.stack_sb -poker.starting_villain)
        print("Difference stack", pos_player[1], poker.stack_bb - poker.starting_hero if poker.position == 1 else poker.stack_bb -poker.starting_villain)


#Model and Agent from: https://github.com/philtabor/Youtube-Code-Repository/blob/master/ReinforcementLearning/DeepQLearning/simple_dqn_tf2.py

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from keras import backend as K

class ReplayBuffer():
    def __init__(self, max_size, input_dims):
        self.mem_size = max_size
        self.mem_cntr = 0

        self.state_memory = np.zeros((self.mem_size, *input_dims), 
                                    dtype=np.float32)
        self.new_state_memory = np.zeros((self.mem_size, *input_dims),
                                dtype=np.float32)
        self.action_memory = np.zeros(self.mem_size, dtype=np.int32)
        self.reward_memory = np.zeros(self.mem_size, dtype=np.float32)
        self.terminal_memory = np.zeros(self.mem_size, dtype=np.int32)

    def store_transition(self, state, action, reward, state_, done):
        index = self.mem_cntr % self.mem_size
        self.state_memory[index] = state
        self.new_state_memory[index] = state_
        self.reward_memory[index] = reward
        self.action_memory[index] = action
        self.terminal_memory[index] = 1 - int(done)
        self.mem_cntr += 1

    def sample_buffer(self, batch_size):
        max_mem = min(self.mem_cntr, self.mem_size)
        batch = np.random.choice(max_mem, batch_size, replace=False)

        states = self.state_memory[batch]
        states_ = self.new_state_memory[batch]
        rewards = self.reward_memory[batch]
        actions = self.action_memory[batch]
        terminal = self.terminal_memory[batch]

        return states, actions, rewards, states_, terminal



def build_dqn(lr, n_actions, input_dims, fc1_dims, fc2_dims):
    model = keras.Sequential()
    model.add(tf.keras.layers.Masking(mask_value=-1., input_shape=input_dims))
    model.add(keras.layers.Dense(fc1_dims))
    #model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.LeakyReLU(alpha=0.05))
    model.add(keras.layers.Dense(fc2_dims))
    #model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.LeakyReLU (alpha=0.05))
    model.add(keras.layers.Dense(n_actions,activation=None))
    #model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Lambda(lambda x: K.tanh(0.1*x)*20)) #the input to this layer shoudl be between -3 and 3, so that the max/min outputs are .995

    model.compile(optimizer=Adam(learning_rate=lr), loss='mean_squared_error')

    return model
class Agent():
    def __init__(self, lr, gamma, n_actions, epsilon, batch_size,
                input_dims, epsilon_dec=1e-4, epsilon_end=0.02,
                mem_size=1000000, fname='HUPoker.h5'):
        self.action_space = [i for i in range(n_actions)]
        self.gamma = gamma
        self.epsilon = epsilon
        self.eps_dec = epsilon_dec
        self.eps_min = epsilon_end
        self.batch_size = batch_size
        self.model_file = fname
        self.memory = ReplayBuffer(mem_size, input_dims)
        self.q_eval = build_dqn(lr, n_actions, input_dims, 51,26)
        self.total_loss = 0 #keep track of the total loss
        self.episodes_learned = 0
    def store_transition(self, state, action, reward, new_state, done):
        self.memory.store_transition(state, action, reward, new_state, done)

    def choose_action(self, observation):
        state = np.array([observation])
        actions = self.q_eval.predict(state)

        if np.random.random() < self.epsilon:
            weights=actions-actions.min()
            action = random.choices(
            population=range(12),
            weights=list(weights[0]),
            k=1)[0]

        else:
            action = np.argmax(actions)


        return action

    def learn(self):
        if self.memory.mem_cntr < self.batch_size:
            return

        states, actions, rewards, states_, dones = \
                self.memory.sample_buffer(len(self.memory.state_memory))

        q_eval = self.q_eval.predict(states)
        q_next = self.q_eval.predict(states_)
        batch_index = np.arange(len(self.memory.state_memory), dtype=np.int32)

        q_target = np.copy(q_eval)


        q_target[batch_index, actions] = rewards + \
                        self.gamma * np.max(q_next, axis=1)*dones
        
        # we know the exact q-values for folds. You lose exactly what you have put in the pot thus far.

        for i in batch_index:
            q_target[i, 0] = states[i, 1] -19.5 if states[i, 0] == 0 else states[i,2] -19
            #sometimes we know that qs for different actions are the same. When villain is all- all qs except for fold are equal.they are all the same as allin, so we update q for all actions.
            # this is the last action everytime, so all get the reward.

            if ((states[i, 0] == 0 and states[i, 2] ==0) or (states[i, 0] == 1 and states[i, 1]==0)):
                q_target[i, 2:] = -20
                if rewards[i] != 0:
                    q_target[i, 1] = rewards[i]
            if states[i,8] == states[i,9]:
                q_target[i, 0] = -20
 

        
        self.q_eval.fit(states, q_target, batch_size=400, epochs=4)


    def save_model(self):
        self.q_eval.save(self.model_file)


    def load_model(self):
        self.q_eval = load_model(self.model_file)

test


2022-01-01 11:50:10.878048: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-01 11:50:10.878089: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import random
#import numpy as np
import itertools
agent="test"
print(agent)

def play(instance):
    import random
    #import numpy as np
    import itertools
    agent="test"
    print(agent)
    from StrengthEvaluator import StrengthEvaluator as sEval

    values = list(range(2,15))
    suits = [0, 1, 2, 3]
    deck = []
    for value in values:
        for suit in suits:
            deck.append((value, suit))

    def deal_preflop(deck,n_players):
        hands = []                
        for i in range(n_players):
            hand = [deck[i], deck[i+n_players]]
            hand.sort()
            hands.append(tuple(hand))
        return hands

    def deal_flop(deck,n_players):
        return (deck[n_players*2+1:n_players*2+4])

    def deal_turn(deck,n_players):
        return deck[n_players*2+5]

    def deal_river(deck,n_players):
        return deck[n_players*2+7]


    #a function which gives the strength of a hand.

    def check_straightflush(hand):
        hand = hand[:]
        hand.sort(reverse = True)
        for start in range(0,3):
            left = 6 - start

            inarow = 1
            current = start
            nxt = start +1
            while inarow + left > 4 and inarow <5:
                if hand[nxt][0] == hand[current][0] or (hand[nxt][0]+1 == hand[current][0] and hand[current][1] != hand[nxt][1]): #if nxt hand is same value or next in line, but not same suit:

                    nxt += 1
                    left -= 1

                elif hand[current][0] != hand[nxt][0]+1:
                    break
                elif hand[current][1] != hand[nxt][1]:
                    break

                else:
                    inarow += 1
                    left -= 1
                    current = nxt
                    nxt = current + 1

            if inarow >= 5:

                return True, hand[start][0]


        if inarow <5:
            straightflush = False
            #check if Ace to 5 straightflush is in hand:

            for suit in [0,1, 2, 3]:
                if (14, suit) in hand and (2,suit) in hand and (3, suit) in hand and (4, suit) in hand and (5, suit) in hand:

                    straightflush = True
                    return True, 5

            if not straightflush:

                return False


    def check_quads(hand):
        handfreq = {}
        for card in hand:
            if card[0] not in handfreq:
                handfreq[card[0]]=1
            else:
                handfreq[card[0]]+= 1

        for hand in handfreq:
            if handfreq[hand]==4:
                quad = hand
                #höchste Beikarte ermitteln
                del(handfreq[hand])
                kicker = 2
                for card in handfreq.keys():
                    if card > kicker:
                        kicker = card
                return True, [quad, kicker]

        return False


    def check_fullhouse(hand):
        handfreq = {}
        for card in hand:
            if card[0] not in handfreq:
                handfreq[card[0]]=1
            else:
                handfreq[card[0]]+= 1
        if 3 in handfreq.values():
                trips = 2
                for hand in handfreq.keys():
                    if handfreq[hand]==3 and hand >trips:
                        trips = hand
                del(handfreq[trips])
                pair = 1
                for hand in handfreq.keys():
                    if handfreq[hand] in (2,3) and hand >pair:
                        pair = hand
                if pair >1:
                    return True, [trips, pair]
        return False

    def check_flush(hand):
        #count suits
        spades = []
        hearts = []
        clubs = []
        diamonds = []
        for card in hand:
            if card[1] ==0:
                spades.append(card[0])
            elif card[1] == 1:
                hearts.append(card[0])
            elif card[1] == 2:
                clubs.append(card[0])
            else: 
                diamonds.append(card[0])


        if len(spades) >=5:
            suit = spades
        elif len(hearts) >= 5:
            suit = hearts
        elif len(clubs) >= 5:
            suit = clubs
        elif len(diamonds)>= 5:
            suit = diamonds
        else:
            return False

        suit.sort(reverse=True)
        return True, suit[0:5]


    def check_straight(hand):
        hand = hand[:]
        hand.sort(reverse = True)
        for start in range(0,3):
            left = 6 - start
            inarow = 1
            current = start
            nxt = start +1
            while inarow + left > 4 and inarow <5:

                if hand[nxt][0] == hand[current][0]:

                    nxt += 1
                    left -= 1

                elif hand[current][0] != hand[nxt][0]+1:
                    break

                else:
                    inarow += 1
                    current = nxt
                    nxt = current + 1
                    left -=1
            if inarow >= 5:
                return True, hand[start][0]

        if inarow <5:
            straight = False
            #check if Ace to 5 straight is in hand:
            values = []
            for card in hand:
                values.append(card[0])
                if 14 in values and 2 in values and 3 in values and 4 in values and 5 in values:

                    straight = True
                    return True, 5

            if not straight:
                return False

    def check_trips(hand):
        handfreq = {}
        for card in hand:
            if card[0] not in handfreq:
                handfreq[card[0]]=1
            else:
                handfreq[card[0]]+= 1
        for value in handfreq:
            if handfreq[value]==3:
                trips = value
                del(handfreq[value])
                cards=[]
                cards.append(trips)
                kickers = list(handfreq.keys())
                kickers.sort(reverse=True)
                cards.extend(kickers[0:2])
                return True, cards
        return False

    def check_twopair(hand):
        handfreq = {}
        for card in hand:
            if card[0] not in handfreq:
                handfreq[card[0]]=1
            else:
                handfreq[card[0]]+= 1

        pairs = []
        highcard = 2
        for value in handfreq: #if freq == 2 append to pairs, else check for highcard

            if handfreq[value]==2:
                pairs.append(value)
            else:
                if value > highcard:
                    highcard = value

        if len(pairs)>=2:
            pairs.sort(reverse=True)
            if len(pairs)>2 and pairs[2]>highcard:
                return True, [pairs[0], pairs[1], pairs[2]]
            return True, [pairs[0], pairs[1], highcard]
        else:
            return False

    def check_pair(hand): #if True return list with pair-card and four strongest non-pair values
        hand = hand[:]
        hand.sort(reverse=True)
        cards = []
        for i in range(6):
            for j in range(i+1,7):

                if hand[i][0] == hand[j][0]:
                    pair = hand[i][0]
                    del(hand[i], hand[i])
                    cards.append(pair)
                    for k in range(3):
                        cards.append(hand[k][0])                                   
                    return (True, cards)
        return False

    def strength(hand): #Using the check hand functions above. Starting with the strongest hand. If hand is True, the function above returns True and the meaningful information about the hand. If not, it creates a
        #TypeError, because only False is retured, so nothing can be written into the output variable. We use this to profit from the try, except statements. This might look weired, but by that we only have to
        #run the functions one to test for the hand and create the output.

        try:
            a, output = check_straightflush(hand)
            return 8, output

        except TypeError:
            try:
                a, output = check_quads(hand)
                return 7, output
            except TypeError:
                try:
                    a, output = check_fullhouse(hand)
                    return 6, output
                except TypeError:
                    try:
                        a, output = check_flush(hand)        
                        return 5, output
                    except TypeError:
                        try:
                            a, output = check_straight(hand)
                            return 4, output
                        except TypeError:
                            try:
                                a, output = check_trips(hand)
                                return 3, output
                            except TypeError:
                                try:
                                    a, output = check_twopair(hand)
                                    return 2, output
                                except TypeError:
                                    try:
                                        a, output = check_pair(hand)
                                        return  1, output
                                    except TypeError:
                                        #if hand is not pair, it's highcard. We return strength of 0 and a sorted list of cardvalues
                                        highcards = []
                                        hand = hand[:]
                                        hand.sort(reverse = True)
                                        for i in range(5):
                                            highcards.append(hand[i][0])
                                        return 0, highcards

    def compare(hands): #takes in a dictonary of hands with position as keys (sb = 0) and returns dictonary of positions with rank (0 is strongest)
        rankings = hands.copy()
        for position in hands:
            rankings[position] = (strength(hands[position]))
        #create a sorted list of tuples with positions and strengths
        sorted_hands = sorted(rankings.items(), key=lambda x: x[1], reverse=True)
        #create a dictonary of positions and ranking. Equal strengths should get same ranking.
        ranking_dict = {sorted_hands[0][0]:0}
        for i in range(1, len(sorted_hands)):
            if sorted_hands[i][1] == sorted_hands[i-1][1]:
                ranking_dict[sorted_hands[i][0]]=i-1
            else:
                ranking_dict[sorted_hands[i][0]]=i
        return ranking_dict

    class HUPoker:   
        def __init__(self, stack_hero, stack_villain,random_stack = False, minstack = 5, maxstack = 200, n_run =20, hero=agent, villain = agent): #we are going to train the AI by playing against itself. Later we are going to implement
            #a user interface, to play against. 
            self.starting_hero = stack_hero
            self.starting_villain = stack_villain
            self.n_run = n_run #we want to reward plays by the aproximate ev, so here we define how many times to board is run after a showdown prior to the river.
            self.hero = hero #the agents that interact with the environment. 
            self.villain = villain
            self.random_stack = random_stack
            self.minstack, self.maxstack  = minstack, maxstack


        def reset(self): #shuffles the deck, seats the two players and returns position, stack sizes, hand (only suited or unsuited, because actual stuits are not important preflop) 
            if self.random_stack:
                self.starting_hero = np.random.randint(self.minstack,self.maxstack) 
                self.starting_villain = np.random.randint(self.minstack,self.maxstack)
            #keep track of action counts
            self.count_round = -1

            self.position = np.random.randint(2)
            if self.position == 0:
                self.stack_sb = self.starting_hero - 0.5 
                self.stack_bb = self.starting_villain - 1
            else:
                self.stack_bb = self.starting_hero - 1 
                self.stack_sb = self.starting_villain - 0.5
            self.bet_sb = 0.5 #keeps track of the bet chips by player in current street
            self.bet_bb = 1
            self.left_bet_round = [1,0] #which players are left to bet in this round
            self.pot = self.bet_sb + self.bet_bb    
            self.street = 0 #0: preflop, 1:flop, 2: turn, 3: river

            random.shuffle(deck)
            self.board_current = []

            self.holecards = deal_preflop(deck,2)[self.position]
            self.holecards_villain = deal_preflop(deck, 2)[0] if self.position == 1 else deal_preflop(deck, 2)[1]
            self.value_low = self.holecards[0][0]
            self.value_high = self.holecards[1][0]
            if self.holecards[0][1] == self.holecards[1][1]:
                self.suited =1
            else:
                self.suited = 0


            self.action_villain = -1 #meaningless value

            #initiate hand history to -1
            self.hh =[[],[],[],[]]
            for street in range(4):
                for i in range(20):
                    self.hh[street].append(-1)

            self.suit_low = -1
            self.suit_high = -2
            self.flop1val = -1 #meaningless values
            self.flop1suit = -1 #meaningless values
            self.flop2val = -1 #meaningless values
            self.flop2suit = -1 #meaningless values
            self.flop3val = -1 #meaningless values
            self.flop3suit = -1 #meaningless values
            self.turnval = -1
            self.turnsuit = -1
            self.riverval = -1
            self.riversuit = -1
            done = False

            #constructing observation for villain to chose action on.
            if self.position == 1:
                position = 0
                value_low = self.holecards_villain[0][0]
                suit_low = self.holecards_villain[0][1]
                value_high = self.holecards_villain[1][0]
                suit_high = self.holecards_villain[1][1]
                suited = 1 if suit_low == suit_high else 0
                if self.street == 0:
                    suit_high, suit_low = -1, -1

                observation_villain = self.create_observation_villain()
                self.action_villain = self.villain.choose_action(observation_villain)

                if self.action_villain == 0:
                    self.stack_bb += 1.5
                    bet = 0
                    done = True

                elif self.action_villain == 1:
                    bet = 0.5

                elif self.action_villain >= 2 and self.action_villain <6:#minraise - at least complete plus the last raised bit
                    bet = 1.5

                elif self.action_villain == 6 or self.action_villain == 7: #2.5 total bet villain
                    bet = 2.5*self.bet_bb - self.bet_sb

                elif self.action_villain == 8 or self.action_villain == 9: #3 total bet villain
                    bet = 3*self.bet_bb - self.bet_sb

                elif self.action_villain == 10: #4 total bet villain
                    bet = 4*self.bet_bb - self.bet_sb

                else: #allin
                    bet = self.stack_sb

                self.bet_sb += bet
                self.stack_sb -= bet
                self.pot += bet
                self.left_bet_round = [1]
                self.hh[0][0] = bet
                self.count_round = 0
            self.create_observation()

            return self.observation, done


        def create_observation(self): #creates the observation array. A state in poker consists of all actions made in the entire hand. So we have to save all actions by hero and villain.
            #the best way to do so is to keep track of all bet sizes. We need enough empty slots to keep track of all bets. A check gets the value 0.
                self.observation = np.asarray([self.position, self.stack_sb, self.stack_bb, self.street, self.value_low, self.value_high, self.suited,  
                                           self.pot, self.bet_sb, self.bet_bb, self.suit_low, self.suit_high, 
                                            self.hh[0][0], self.hh[0][1], self.hh[0][2], self.hh[0][3],self.hh[0][4], self.hh[0][5], self.hh[0][6],self.hh[0][7], self.hh[0][8], self.hh[0][9],
                                            self.hh[0][10], self.hh[0][11], self.hh[0][12], self.hh[0][13],self.hh[0][14], self.hh[0][15], self.hh[0][16],self.hh[0][17], self.hh[0][18], self.hh[0][19], 
                                            self.hh[1][0], self.hh[1][1], self.hh[1][2], self.hh[1][3],self.hh[1][4], self.hh[1][5], self.hh[1][6],self.hh[1][7], self.hh[1][8], self.hh[1][9],
                                            self.hh[1][10], self.hh[1][11], self.hh[1][12], self.hh[1][13],self.hh[1][14], self.hh[1][15], self.hh[1][16],self.hh[1][17], self.hh[1][18], self.hh[1][19], 
                                            self.hh[2][0], self.hh[2][1], self.hh[2][2], self.hh[2][3],self.hh[2][4], self.hh[2][5], self.hh[2][6],self.hh[2][7], self.hh[2][8], self.hh[2][9],
                                            self.hh[2][10], self.hh[2][11], self.hh[2][12], self.hh[2][13],self.hh[2][14], self.hh[2][15], self.hh[2][16],self.hh[2][17], self.hh[2][18], self.hh[2][19],
                                            self.hh[3][0], self.hh[3][1], self.hh[3][2], self.hh[3][3],self.hh[3][4], self.hh[3][5], self.hh[3][6],self.hh[3][7], self.hh[3][8], self.hh[3][9],
                                            self.hh[3][10], self.hh[3][11], self.hh[3][12], self.hh[3][13],self.hh[3][14], self.hh[3][15], self.hh[3][16],self.hh[3][17], self.hh[3][18], self.hh[3][19],
                                           self.flop1val, self.flop1suit, self.flop2val, self.flop2suit, self.flop3val, self.flop3suit, 
                                               self.turnval, self.turnsuit, self.riverval, self.riversuit]).astype(np.float32)

        def create_observation_villain(self): #this is the last action by hero. If two betting rounds in a row, set to -1
            position = 0 if self.position == 1 else 1
            value_low = self.holecards_villain[0][0]
            suit_low = self.holecards_villain[0][1] if self.street != 0 else -1
            value_high = self.holecards_villain[1][0]
            suit_high = self.holecards_villain[1][1] if self.street != 0 else -1
            suited = 1 if suit_low == suit_high else 0


            return np.asarray([position, self.stack_sb, self.stack_bb, self.street, value_low, value_high, suited,  
                                       self.pot, self.bet_sb, self.bet_bb, suit_low, suit_high,
                                       self.hh[0][0], self.hh[0][1], self.hh[0][2], self.hh[0][3],self.hh[0][4], self.hh[0][5], self.hh[0][6],self.hh[0][7], self.hh[0][8], self.hh[0][9],
                                        self.hh[0][10], self.hh[0][11], self.hh[0][12], self.hh[0][13],self.hh[0][14], self.hh[0][15], self.hh[0][16],self.hh[0][17], self.hh[0][18], self.hh[0][19], 
                                        self.hh[1][0], self.hh[1][1], self.hh[1][2], self.hh[1][3],self.hh[1][4], self.hh[1][5], self.hh[1][6],self.hh[1][7], self.hh[1][8], self.hh[1][9],
                                        self.hh[1][10], self.hh[1][11], self.hh[1][12], self.hh[1][13],self.hh[1][14], self.hh[1][15], self.hh[1][16],self.hh[1][17], self.hh[1][18], self.hh[1][19], 
                                        self.hh[2][0], self.hh[2][1], self.hh[2][2], self.hh[2][3],self.hh[2][4], self.hh[2][5], self.hh[2][6],self.hh[2][7], self.hh[2][8], self.hh[2][9],
                                        self.hh[2][10], self.hh[2][11], self.hh[2][12], self.hh[2][13],self.hh[2][14], self.hh[2][15], self.hh[2][16],self.hh[2][17], self.hh[2][18], self.hh[2][19],
                                        self.hh[3][0], self.hh[3][1], self.hh[3][2], self.hh[3][3],self.hh[3][4], self.hh[3][5], self.hh[3][6],self.hh[3][7], self.hh[3][8], self.hh[3][9],
                                        self.hh[3][10], self.hh[3][11], self.hh[3][12], self.hh[3][13],self.hh[3][14], self.hh[3][15], self.hh[3][16],self.hh[3][17], self.hh[3][18], self.hh[3][19],
                                        self.flop1val, self.flop1suit, self.flop2val, self.flop2suit, self.flop3val, self.flop3suit, 
                                        self.turnval, self.turnsuit, self.riverval, self.riversuit]).astype(np.float32)

        def implement_action(self, action, player = 'hero'): #implements either heros or villains action to the point where either the next player is to act or the hand is finished
            bet = 0
            self.count_round += 1
            done =False
            next_to_act = self.left_bet_round.pop() #last in list acts first. BB acts last preflop, which we allready tool care of above. Otherwise BB acts first.
            #transform action into legal option
            if action == 0:
                if self.bet_sb == self.bet_bb: #player cannot fold, if he can check
                    action = 1

            elif action > 1: #player cannot raise, if he has less than pot - self.bet
                if (player == 'hero' and self.position == 0 and self.bet_bb - self.bet_sb >= self.stack_sb) or (
                    player == 'hero' and self.position == 1 and self.bet_sb - self.bet_bb >= self.stack_bb) or(
                    player == 'villain' and self.position == 1 and self.bet_bb - self.bet_sb >= self.stack_sb) or (
                    player == 'villain' and self.position == 0 and self.bet_sb - self.bet_bb >= self.stack_bb):
                    action = 1


            if action == 0: #player folds. Action is already transformed into 1, if player is first in.
                if next_to_act == 0: #hero was sb
                    self.stack_bb += self.pot

                else: #player is bb
                    self.stack_sb += self.pot
                self.left_bet_round = []
                done = True #round is finished. self.reset should be performed outside this function
                self.hh[self.street][self.count_round] = bet
                done=True
            elif action == 1: #player calls/checks
                if next_to_act == 0: #player is sb
                    if self.bet_sb == self.bet_bb: #bets are ballanced, so player can check.

                        pass               

                    else: #bets are unbalanced, so player calls
                        left_to_bet = self.bet_bb - self.bet_sb
                        if left_to_bet < self.stack_sb: #hero doesn't have to go allin to call
                            bet = self.bet_bb-self.bet_sb
                            self.bet_sb = self.bet_bb
                            self.stack_sb -= left_to_bet
                            self.pot += left_to_bet

                        else: #player goes allin
                            self.pot += self.stack_sb
                            self.bet_sb = self.bet_sb + self.stack_sb
                            bet = self.stack_sb
                            self.stack_sb = 0
                            #give bb chips higher than sb_stack back
                            bet_difference = self.bet_bb - self.bet_sb
                            self.stack_bb += bet_difference
                            self.pot -= bet_difference
                            #self.hh[self.street][self.count_round] = bet
                            self.showdown()
                            done = True


                else: #player is bb
                    if self.bet_sb == self.bet_bb: #bets are ballanced, so player can check.
                        pass


                    else: #bets are unbalanced, so player calls
                        left_to_bet = self.bet_sb - self.bet_bb
                        if left_to_bet <= self.stack_bb: #hero doesn't have to go allin to call
                            bet = self.bet_sb -self.bet_bb
                            self.bet_bb = self.bet_sb
                            self.stack_bb -= left_to_bet
                            self.pot += left_to_bet
                            self.left_bet_round = []


                        else: #plyer goes allin 
                            self.pot += self.stack_bb
                            self.bet_bb = self.bet_bb + self.stack_bb
                            bet = self.stack_bb
                            self.stack_bb = 0
                            #give bb chips higher than bb_stack back
                            bet_difference = self.bet_sb - self.bet_bb
                            self.stack_sb += bet_difference
                            self.pot -= bet_difference


                            #self.hh[self.street][self.count_round] = bet
                            self.showdown()
                            done = True


            else: #player raises/bets From here on we have to change the code.
                # if unbet the bet variable indicates how much the player will increase his current betsize.
                if next_to_act == 0 and self.bet_bb == 0 or next_to_act == 1 and self.bet_sb == 0:
                    if action == 2:
                        bet = 1
                    elif action == 3:
                        bet = self.pot//10.0
                        if bet < 1: #bet has to be at least 1BB
                            bet = 1
                    elif action == 4:
                        bet = self.pot//4.0
                        if bet < 1: #bet has to be at least 1BB
                            bet = 1
                    elif action ==5:
                        bet = self.pot//3.0
                        if bet < 1: #bet has to be at least 1BB
                            bet = 1

                    elif action == 6:
                        bet = self.pot//2.0
                    elif action ==7:
                        bet = 3*self.pot//4.0
                    elif action == 8:
                        bet = self.pot
                    elif action == 9:
                        bet = self.pot*13//10.0
                    elif action == 10:
                        bet = self.pot*2

                    else:
                        bet = self.stack_sb if self.position == 0 else self.stack_bb

                #if pot is bet:
                else:

                    if action == 2 or action == 3:#minraise - at least complete plus the last raised bit
                        if next_to_act == 0:
                            #preflop first in
                            if self.street == 0 and self.bet_sb == 0.5:
                                bet = 1.5
                            else:
                                bet = 2*(self.bet_bb - self.bet_sb)
                        else:
                            bet = 2*(self.bet_sb - self.bet_bb)
                            if bet < 1:
                                bet = 1#at least a bet of 1
                    elif action == 4 or action == 5: # 2 total bet villain
                        if next_to_act == 0:
                            bet = 2*self.bet_bb - self.bet_sb
                        else:
                            bet = 2*self.bet_sb - self.bet_bb
                    elif action == 6 or action == 7: #2.5 total bet villain
                        if next_to_act == 0:
                            bet = 2.5*self.bet_bb - self.bet_sb
                        else:
                            bet = 2.5*self.bet_sb - self.bet_bb

                    elif action == 8 or action == 9: #3 total bet villain
                        if next_to_act == 0:
                            bet = 3*self.bet_bb - self.bet_sb
                        else:
                            bet = 3*self.bet_sb - self.bet_bb
                    elif action == 10: #4 total bet villain
                        if next_to_act == 0:
                            bet = 4*self.bet_bb - self.bet_sb
                        else:
                            bet = 4*self.bet_sb - self.bet_bb

                    else: #allin
                        bet = self.stack_sb if next_to_act == 0 else self.stack_bb

                if (bet >= self.stack_sb and next_to_act == 0) or (bet >= self.stack_bb and next_to_act == 1): #allin if not enough chips left.
                    bet = self.stack_sb if next_to_act == 0 else self.stack_bb



                if next_to_act == 0: #player is sb
                    self.bet_sb += bet
                    self.pot += bet
                    self.stack_sb -= bet
                    self.left_bet_round = [1] if self.stack_bb != 0 else [] #on an agressive move, his opponent is next to act. If player went all in, no further action.

                else:
                    self.bet_bb += bet
                    self.pot += bet
                    self.stack_bb -= bet
                    self.left_bet_round = [0] if self.stack_sb != 0 else []


            if len(self.left_bet_round) == 0:
                if not done:
                    self.bet_sb = 0 #put bet in round back to 0
                    self.bet_bb = 0

                    if self.street == 3 or self.stack_sb == 0 or self.stack_bb == 0:
                        self.hh[self.street][self.count_round] = bet
                        self.showdown()
                        done = True
                        return done

                    if self.street == 0:
                        self.hh[self.street][self.count_round] = bet
                        self.street = 1
                        self.count_round =-1

                        self.suit_low = self.holecards[0][1]
                        self.suit_high = self.holecards[1][1]
                        flop = deal_flop(deck,2)
                        self.board_current = flop
                        self.flop1val = flop[0][0]
                        self.flop1suit = flop[0][1]
                        self.flop2val = flop[1][0]
                        self.flop2suit = flop[1][1]
                        self.flop3val = flop[2][0]
                        self.flop3suit = flop[2][1]

                        self.left_bet_round = [0,1]

                        self.create_observation()
                        return done

                    elif self.street ==1:
                        self.hh[self.street][self.count_round] = bet
                        self.street = 2
                        self.count_round = -1
                        turn = deal_turn(deck,2)
                        self.board_current.append(turn)

                        self.turnval = turn[0]
                        self.turnsuit = turn[1]

                        self.left_bet_round = [0,1]


                        self.create_observation()
                        return done

                    else:
                        self.hh[self.street][self.count_round] = bet
                        self.street = 3
                        self.count_round = -1
                        river = deal_river(deck,2)
                        self.board_current.append(river)
                        self.riverval = river[0]
                        self.riversuit = river[1]

                        self.left_bet_round = [0,1]


                        self.create_observation()
                        return done

            self.hh[self.street][self.count_round] = bet
            return done       


        def step(self, action): #implements hero's action all up to the point where either the hand is over, or the next action is asked
            done = self.implement_action(action, 'hero')
            self.action_villain = -1 #if villain has no action after hero, set back to no action. Meaning that hero bets two times in a row
            if not done and self.left_bet_round != []:
                if self.left_bet_round[-1] != self.position: #villain is next to act.
                    #constructing observation for villain to chose action on.            
                    observation_villain = self.create_observation_villain()
                    self.action_villain = self.villain.choose_action(observation_villain)  
                    done = self.implement_action(self.action_villain,'villain')

                       #check if another villains action must be implemented.
                    if not done and self.left_bet_round != []: #player left in round.
                        if self.left_bet_round[-1] != self.position: #next to act is not hero.
                        #check if we have to implement another villains action. 
                            observation_villain = self.create_observation_villain()
                            self.action_villain = self.villain.choose_action(observation_villain)            
                            done = self.implement_action(self.action_villain,'villain')
            self.create_observation()
            return done







        def showdown(self):

            if self.street == 0:
                cards_left = 5 #how many cards left to deal
            else:
                cards_left = 3 - self.street

            share_pot_sb, share_pot_bb = 0, 0
            for i in range(self.n_run):

                board = self.board_current[:]


                board_remaining = random.sample(deck[8:],cards_left) #we don't have to take into accounts the cards already dealt, because we start with the river card anyway.
                board.extend(board_remaining)
                #self.board_current.extend(board_remaining)


                hand_hero_total = list(self.holecards) + board
                hand_villain_total=list(self.holecards_villain) + board

                comparison = compare({0:hand_hero_total, 1:hand_villain_total})
                if comparison[0] < comparison[1]:

                    if self.position == 0:                    
                        share_pot_sb += self.pot
                    else:
                        share_pot_bb += self.pot
                elif comparison[0] == comparison[1]:

                    share_pot_sb += self.pot/2
                    share_pot_bb += self.pot/2

                else:

                    if self.position == 0:
                        share_pot_bb += self.pot
                    else:
                        share_pot_sb += self.pot


            self.stack_sb += share_pot_sb/self.n_run

            self.stack_bb += share_pot_bb/self.n_run

            self.create_observation()
            done = True
            #return done

    def play_against(n_games = 100000,hero = agent, villain=agent):
        hero = hero
        villain = villain
        env = HUPoker(100,100, hero=hero, villain = villain)
        total_score = 0

        for i in range(n_games):
            observation, done = env.reset()
            reward = 0

            while not done:
                action = hero.choose_action(observation)
                done = env.step(action)
                observation = env.observation



            reward = env.stack_sb - env.starting_hero if env.position == 0 else env.stack_bb - env.starting_hero 

            total_score += reward
            avg_score = total_score/(i+0.00001)
            print('episode: ', i, 'reward %.2f' % reward,
                            'average_score %.2f' % avg_score,
                            'total score %.2f' % total_score)
        return avg_score, env.observation

    class PokerInterface():
        def __init__(self, hero=agent, villain = agent):
            self.hero = hero
            self.villain = villain
            self.poker = HUPoker(100,100, hero= self.hero, villain = self.villain)


        def simulate(self):
            poker = self.poker
            observation = poker.reset()
            hand = poker.holecards
            pos_player = ['hero','villain'] if poker.position == 0 else ['villain','hero']

            value_dict = {10: 'T', 11: 'J', 12: 'Q', 13: 'K', 14: 'A'}
            for i in range (2,10):
                value_dict[i] = str(i)
            suit_dict = {0 : '\u2660', 1:'\u2665', 2: '\u2663', 3: '\u2666'} #creating suits from unicode
            dict_action = {0: 'folds', 1: 'checks/calls', 2: 'minbets/raises', 3: 'bets/raises', 4: 'bets/raises', 5: 'bets/raises', 6: 'bets/raises', 7: 'bets/raises',
                           8: 'bets/raises', 9: 'bets/raises', 10: 'bets/raises', 11: 'goes all in',}

            card1 = value_dict[hand[0][0]]+suit_dict[hand[0][1]]
            card2 = value_dict[hand[1][0]]+suit_dict[hand[1][1]]
            print(pos_player[0], "is the button")            
            print("Stack Hero ", poker.starting_hero)
            print("Stack Villain", poker.starting_villain)
            print(pos_player[0], 'posts small blind 0.5')
            print(pos_player[1], 'posts big blind 1')
            print("*** HOLE CARDS ***")
            print("Dealt to Hero ", card1, card2)

            done = False
            if poker.position == 1:
                action = poker.action_villain
                print('Villain', dict_action[action])
                if action == 0:
                    done = True
                poker.left_bet_round = [1]

            next_to_act = 'hero'            


            while not done:
                if poker.bet_sb == poker.bet_bb:
                    next_to_act = 'hero' if poker.position == 1 else 'villain'
                street = poker.street
                if street != 0:
                    print("Stack Hero ", poker.stack_sb if poker.position == 0 else poker.stack_bb)
                    print("Stack Villain ", poker.stack_sb if poker.position == 1 else poker.stack_bb)
                    board = ""
                    for i in range(len(poker.board_current)):
                                board += value_dict[poker.board_current[i][0]]+suit_dict[poker.board_current[i][1]]
                    if street == 1:
                        print("*** FLOP ***", board)

                    elif street == 2:
                        print("*** TURN ***", board[:-2], " ", board[-2:])

                    else:
                        print("*** RIVER ***", board[:-2], " ", board[-2:])

                while street == poker.street and not done:
                    if next_to_act == 'hero':
                        poker.create_observation()
                        observation = poker.observation
                        action = self.hero.choose_action(observation)

                    else:

                        observation_villain = poker.create_observation_villain()
                        action = self.villain.choose_action(observation_villain)


                    done = poker.implement_action(action, player = next_to_act)
                    print("done =", done)
                    print(next_to_act, action, dict_action[action])
                    next_to_act = 'villain' if next_to_act == 'hero' else 'hero'

            if action != 0: #if noone folded
                print("*** SHOW DOWN ***")
                card1_villain = value_dict[poker.holecards_villain[0][0]]+suit_dict[poker.holecards_villain[0][1]]
                card2_villain = value_dict[poker.holecards_villain[1][0]]+suit_dict[poker.holecards_villain[1][1]]
                if poker.position == 0:
                    print(pos_player[0], "shows", card1, card2 )
                    print(pos_player[1], "shows", card1_villain, card2_villain)
                else:
                    print(pos_player[0], "shows", card1_villain, card2_villain)
                    print(pos_player[1], "shows", card1, card2 )

                board = ""
                for i in range(len(poker.board_current)):
                            board += value_dict[poker.board_current[i][0]]+suit_dict[poker.board_current[i][1]]
                if street < 1:
                    print("*** FLOP ***", board[:6])

                if street < 2:
                    print("*** TURN ***", board[:6], " ", board[6:-2])

                if street < 3:
                    print("*** RIVER ***", board[:-2], " ", board[-2:])

            if (poker.stack_sb > poker.starting_hero and poker.position == 0) or poker.stack_bb > poker.starting_hero and poker.position == 1:
                print("Hero wins", poker.pot)
            elif poker.stack_sb == poker.starting_hero or poker.stack_bb == poker.starting_hero:
                print("Split pot. Bot players receive", poker.pot/2)
            else:
                print("Villain wins", poker.pot)

            print("*** SUMMARY ***")
            print("Difference stack", pos_player[0], poker.stack_sb - poker.starting_hero if poker.position == 0 else poker.stack_sb -poker.starting_villain)
            print("Difference stack", pos_player[1], poker.stack_bb - poker.starting_hero if poker.position == 1 else poker.stack_bb -poker.starting_villain)


  #Model and Agent from: https://github.com/philtabor/Youtube-Code-Repository/blob/master/ReinforcementLearning/DeepQLearning/simple_dqn_tf2.py

    import numpy as np
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.models import load_model
    from keras import backend as K

    class ReplayBuffer():
        def __init__(self, max_size, input_dims):
            self.mem_size = max_size
            self.mem_cntr = 0

            self.state_memory = np.zeros((self.mem_size, *input_dims), 
                                        dtype=np.float32)
            self.new_state_memory = np.zeros((self.mem_size, *input_dims),
                                    dtype=np.float32)
            self.action_memory = np.zeros(self.mem_size, dtype=np.int32)
            self.reward_memory = np.zeros(self.mem_size, dtype=np.float32)
            self.terminal_memory = np.zeros(self.mem_size, dtype=np.int32)

        def store_transition(self, state, action, reward, state_, done):
            index = self.mem_cntr % self.mem_size
            self.state_memory[index] = state
            self.new_state_memory[index] = state_
            self.reward_memory[index] = reward
            self.action_memory[index] = action
            self.terminal_memory[index] = 1 - int(done)
            self.mem_cntr += 1

        def sample_buffer(self, batch_size):
            max_mem = min(self.mem_cntr, self.mem_size)
            batch = np.random.choice(max_mem, batch_size, replace=False)

            states = self.state_memory[batch]
            states_ = self.new_state_memory[batch]
            rewards = self.reward_memory[batch]
            actions = self.action_memory[batch]
            terminal = self.terminal_memory[batch]

            return states, actions, rewards, states_, terminal



    def build_dqn(lr, n_actions, input_dims, fc1_dims, fc2_dims):
        model = keras.Sequential()
        model.add(tf.keras.layers.Masking(mask_value=-1., input_shape=input_dims))
        model.add(keras.layers.Dense(fc1_dims))
        #model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.LeakyReLU(alpha=0.05))
        model.add(keras.layers.Dense(fc2_dims))
        #model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.LeakyReLU (alpha=0.05))
        model.add(keras.layers.Dense(n_actions,activation=None))
        #model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Lambda(lambda x: K.tanh(0.1*x)*20)) #the input to this layer shoudl be between -3 and 3, so that the max/min outputs are .995

        model.compile(optimizer=Adam(learning_rate=lr), loss='mean_squared_error')

        return model
    class Agent():
        def __init__(self, lr, gamma, n_actions, epsilon, batch_size,
                    input_dims, epsilon_dec=1e-4, epsilon_end=0.02,
                    mem_size=1000000, fname='HUPoker.h5'):
            self.action_space = [i for i in range(n_actions)]
            self.gamma = gamma
            self.epsilon = epsilon
            self.eps_dec = epsilon_dec
            self.eps_min = epsilon_end
            self.batch_size = batch_size
            self.model_file = fname
            self.memory = ReplayBuffer(mem_size, input_dims)
            self.q_eval = build_dqn(lr, n_actions, input_dims, 51,26)
            self.total_loss = 0 #keep track of the total loss
            self.episodes_learned = 0
        def store_transition(self, state, action, reward, new_state, done):
            self.memory.store_transition(state, action, reward, new_state, done)

        def choose_action(self, observation):
            state = np.array([observation])
            actions = self.q_eval.predict(state)
            
            if np.random.random() < self.epsilon:
                #half the time choose action by weight
                if np.random.random() < 0.5:
                    weights=actions-actions.min()
                    action = random.choices(
                    population=range(12),
                    weights=list(weights[0]),
                    k=1)[0]
                #half the time chose a random action
                else:
                    if observation[8] != observation[9]: # check if bets are eqal, so fold is not tried.
                        action = np.random.randint(0,3) #make fold, call/check and bet/raise equally likely
                        if action == 2:
                            action = np.random.randint(2,12)
                    else: #bets are balanced, so fold should not be chosen.
                        action = np.random.randint(1,3)
                        if action ==2:
                            action = np.random.randint(2,12)
                
            else:
                action = np.argmax(actions)
                                   

            return action

        def learn(self):
            if self.memory.mem_cntr < self.batch_size:
                return

            states, actions, rewards, states_, dones = \
                    self.memory.sample_buffer(len(self.memory.state_memory))

            q_eval = self.q_eval.predict(states)
            q_next = self.q_eval.predict(states_)


            q_target = np.copy(q_eval)
            batch_index = np.arange(len(self.memory.state_memory), dtype=np.int32)

            q_target[batch_index, actions] = rewards + \
                            self.gamma * np.max(q_next, axis=1)*dones


            self.q_eval.fit(states, q_target, batch_size=32)


        def save_model(self):
            self.q_eval.save(self.model_file)


        def load_model(self):
            self.q_eval = load_model(self.model_file)

    import time
    inst_fin=[]
    #for i in range(16):
     #   inst_fin.append(0)
    #np.save('inst_fin',np.array(inst_fin))


    agent_1= Agent(gamma=1, epsilon=0.2, lr=0.1,
            input_dims=(104,),
            n_actions=12, mem_size=200, batch_size=64,
            epsilon_end=0.05,
            fname='PokerNew.h5')
    agent_vil= Agent(gamma=1, epsilon=0, lr=0.01, 
            input_dims=(102,),
            n_actions=12, mem_size=2, batch_size=64,
            epsilon_end=0,
            fname='PokerMasked.h5')
    try:
        agent_1.load_model()
        agent_vil.load_model()
    except:
        pass

    env = HUPoker(20,20, hero=agent_1, villain = agent_vil)
    n_rounds = 1
    env.reset()


    for i in range(n_rounds):
        agent_1.memory = ReplayBuffer(200, (102,))
        #try:
         #   agent_1.load_model()
          #  agent_vil.load_model()
        #except:
         #   pass
        #env = HUPoker(100,100, hero=agent_1, villain = agent_vil)
        while agent_1.memory.mem_cntr < 200:
            #try:
             #   agent_1.load_model()
              #  agent_vil.load_model()
            #except:
             #   pass
            done =True
            score = 0
            while done:
                observation, done = env.reset()
                hand = env.holecards
                #add handstrength eval to observation
            while not done:

                error = False
                action = agent_1.choose_action(observation)

                try: 
                    done = env.step(action) #sometimes the chosen actions lead to too many moves on a street (over 20) This happens when players both chose to engage in minraise battles.
                                            #we want to severely punish this behavior.
                except IndexError: #error happens when more than 20 actions are attepted on a single street.
                    done = True
                    error = True

                observation_ = env.observation
                if not done:
                    reward = 0
                else:
                    if error:
                        reward = -20
                    else:
                        reward = 0.5+env.stack_sb - env.starting_hero if env.position == 0 else 1+env.stack_bb - env.starting_hero #to take into account the blinds are posted anyway regardless of action.
                score += reward
                agent_1.store_transition(observation, action, reward, observation_, done)
                observation = observation_



        np.save('Memory/state_memory'+str(instance), agent_1.memory.state_memory[:200])
        np.save('Memory/new_state_memory'+str(instance), agent_1.memory.new_state_memory[:200])
        np.save('Memory/reward_memory'+str(instance), agent_1.memory.reward_memory[:200])
        np.save('Memory/action_memory'+str(instance), agent_1.memory.action_memory[:200])
        np.save('Memory/terminal_memory'+str(instance), agent_1.memory.terminal_memory[:200])
        np.save('Memory/mem_cntr'+str(instance), np.array(agent_1.memory.mem_cntr))
        agent_1.memory.mem_cntr =0


test


In [3]:
import ipyparallel as ipp
cluster = ipp.Cluster(n=8)
await cluster.start_cluster() # or cluster.start_cluster_sync() without await

Using existing profile dir: 'C:\\Users\\dario\\.ipython\\profile_default'
Starting 8 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


In [4]:
rc = cluster.connect_client_sync()
rc.wait_for_engines(n=8)

100%|████████████████████████████████████████████████████████████████████████████████| 8/8 [00:07<00:00,  1.03engine/s]


In [ ]:
import os
import numpy as np
import time
import gc

agent_learn= Agent(gamma=1, epsilon=0, lr=0.02,
        input_dims=(102,),
        n_actions=12, mem_size=1600, batch_size=64,
        epsilon_end=0.05,
        fname='PokerMasked.h5')
try:
    agent_learn.load_model()
except:
    pass


for j in range (20000):
    print (j)
    rc[:].map_sync(play,range(8))
    #inst_fin = np.load('inst_fin.npy')
    

    #while inst_fin.sum()!=13:
       # time.sleep(3)
        #inst_fin = np.load('inst_fin.npy')

    states = []
    states_=[]
    rewards=[]
    actions=[]
    terminals=[]
    for i in range(8):
        state_memory = np.load('Memory/state_memory'+str(i)+'.npy')
        new_state_memory = np.load('Memory/new_state_memory'+str(i)+'.npy')
        reward_memory = np.load('Memory/reward_memory'+str(i)+'.npy')
        action_memory = np.load('Memory/action_memory'+str(i)+'.npy')
        terminal_memory = np.load('Memory/terminal_memory'+str(i)+'.npy')

        states.append(state_memory)
        states_.append(new_state_memory)
        rewards.append(reward_memory)
        actions.append(action_memory)
        terminals.append(terminal_memory)
    
    state_memory=np.concatenate(states,axis=0)
    new_state_memory = np.concatenate(states_,axis=0)
    reward_memory = np.concatenate(rewards,axis=0)
    action_memory = np.concatenate(actions,axis=0)
    terminal_memory = np.concatenate(terminals,axis=0)
    
    agent_learn.memory.state_memory = state_memory
    agent_learn.memory.new_state_memory = new_state_memory
    agent_learn.memory.reward_memory = reward_memory
    agent_learn.memory.action_memory = action_memory
    agent_learn.memory.terminal_memory = terminal_memory
    agent_learn.memory.mem_cntr = 1600
    
    r1 = np.random.randint(0, 1600)
    r2 = np.random.randint(0, 1600)
    observation = agent_learn.memory.state_memory[r1]
    print(observation[0:6],agent_learn.q_eval.predict(np.array([observation])))
    observation = agent_learn.memory.state_memory[r2]
    print(observation[0:6],agent_learn.q_eval.predict(np.array([observation])))

 
    agent_learn.learn()
    

    agent_learn.save_model()
    gc.collect()

Train neural network to estimate hand strength on river

create train data. Simulate hands, output wins, losses and splitpots

In [ ]:
import pandas as pd
import itertools
combos = itertools.combinations(deck,2)
for comb in combos:
    print(comb)
deck_left = set(deck)- set(comb)

In [ ]:
deck_left = set(deck)- set(comb)
len(deck_left)

In [ ]:
pf =deal_preflop(deck,1)
pf

In [ ]:
# let's start simply with preflop only. Here we are going to calculate handstrengths directly.
import pandas as pd
import itertools
combos = itertools.combinations(deck,2)
simHands = pd.DataFrame(columns = ['val1', 'suit1', 'val2', 'suit2', 'nwin', 'nloose', 'nsplit'])
i= 0
for comb in combos:
    deck_left = list(set(deck)- set(comb))
    pf =[comb]
    nwin = 0
    nloose = 0
    nsplit = 0
    n = 5000
    for j in range(n):
        boardAndVil = random.sample(deck_left[2:], 7)
        handhero = list(pf[0])
        handvillain = boardAndVil
        board = boardAndVil[2:7]
        handhero.extend(board)
        comp = compare({0: handhero, 1: handvillain})
        if comp[0] == comp[1]:
            nsplit +=1
        elif comp[0] < comp[1]:
            nwin += 1
        else:
            nloose +=1
    nwin /= n
    nloose /= n
    nsplit /= n
    simHands = simHands.append({'val1': pf[0][0][0],'suit1':pf[0][0][1],'val2':pf[0][1][0],'suit2':pf[0][1][1], 'nwin':nwin, 'nloose':nloose, 'nsplit':nsplit}, ignore_index=True)
    i+=1
    if i%10 ==0:
        print(i)
simHands

Let's agregate all equivalent hands. It's easiest if we sort the dataframe first.

In [ ]:
simHands = simHands.sort_values(by =['val1', 'val2'])
simHands.to_csv('simHandsPreflop.csv', index=False)

In [ ]:
simHands['nsplit'].mean()

Let's always make val1 <= val2

In [ ]:
for i in range(len(simHands)):
    if simHands.loc[i, 'val1'] > simHands.loc[i, 'val2']:
        simHands.loc[i, 'val1'], simHands.loc[i, 'val2'] = simHands.loc[i, 'val2'], simHands.loc[i, 'val1']

In [ ]:
simHandsOffSuitedGrouped = simHands[simHands['suit1']!=simHands['suit2']].groupby(['val1', 'val2'],as_index=False).agg({'nwin':'mean', 'nloose':'mean', 'nsplit':'mean'})
simHandsOffSuitedGrouped[:100]

In [ ]:
simHandsOffSuitedGrouped['suited']=0
simHandsOffSuitedGrouped

In [ ]:
simHandsSuitedGrouped = simHands[simHands['suit1']==simHands['suit2']].groupby(['val1', 'val2'],as_index=False).agg({'nwin':'mean', 'nloose':'mean', 'nsplit':'mean'})
simHandsSuitedGrouped[:100]

In [ ]:
simHandsSuitedGrouped['suited']=1
simHandsSuitedGrouped

In [ ]:
strengthPreflop = simHandsOffSuitedGrouped.append(simHandsSuitedGrouped,ignore_index=True)
strengthPreflop[['val1','val2']]=strengthPreflop[['val1','val2']].astype(int)

strengthPreflop.to_csv('strengthPreflop.csv', index=False)


Let's create a dicotnary and save it, so we can use it us a quick hashable lookuptable later.

In [ ]:
import csv
strengthPreflop = pd.read_csv('strengthPreflop.csv', index_col=False)
lookupPreflop = {}
for i in strengthPreflop.index:
    lookupPreflop[(strengthPreflop.loc[i, 'val1'], strengthPreflop.loc[i, 'val2'],strengthPreflop.loc[i, 'suited'])]=(strengthPreflop.loc[i, 'nwin'], strengthPreflop.loc[i, 'nloose'])

# open file for writing, "w" is writing
w = csv.writer(open("lookupPreflop.csv", "w"))

# loop over dictionary keys and values
for key, val in lookupPreflop.items():
    
    # write every key and value to file
    w.writerow([key, val])

Let's simulate flops. Generated on another computer.

In [62]:
import pandas as pd
simHandsFlop = pd.read_csv('simHandsFlop.csv', index_col=0) 
simHandsFlop = simHandsFlop[['val1', 'suit1','val2','suit2', 'f1val', 'f1suit', 'f2val', 'f2suit' ,'f3val','f3suit','nwin', 'nloose', 'nsplit']]
simHandsFlop

,val1,suit1,val2,suit2,f1val,f1suit,f2val,f2suit,f3val,f3suit,nwin,nloose,nsplit
0,4,2,8,2,2,3,9,3,10,1,0.2120,0.7422,0.0458
1,5,2,10,3,8,3,11,2,13,0,0.3080,0.6248,0.0672
2,12,1,13,3,10,0,10,2,12,2,0.7650,0.2038,0.0312
3,7,0,10,3,2,1,11,0,13,1,0.3252,0.6280,0.0468
4,10,2,14,3,2,2,7,1,13,2,0.5306,0.4516,0.0178
...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,11,3,12,1,5,1,5,2,9,3,0.4462,0.5020,0.0518
79996,6,2,7,2,2,3,10,3,12,3,0.1470,0.7866,0.0664
79997,12,0,14,1,6,2,8,2,10,0,0.4640,0.5140,0.0220
79998,8,0,13,1,4,2,9,3,10,3,0.3910,0.5816,0.0274


Let's create a sequential neural network

In [ ]:
input_dims = 10
fc1_dims = 256
fc2_dims = 128
fc3_dims = 64
fc4_dims = 32
fc5_dims = 16
fc6_dims = 8
fc7_dims = 4
lr = 0.001
model = keras.Sequential()
model.add(keras.layers.Dense(fc1_dims, input_dim=input_dims))
model.add(keras.layers.LeakyReLU (alpha=0.05))
model.add(keras.layers.Dense(fc2_dims))
model.add(keras.layers.LeakyReLU (alpha=0.05))
model.add(keras.layers.Dense(fc3_dims))
model.add(keras.layers.LeakyReLU (alpha=0.05))
model.add(keras.layers.Dense(fc4_dims))
model.add(keras.layers.LeakyReLU (alpha=0.05))
model.add(keras.layers.Dense(fc5_dims))
model.add(keras.layers.LeakyReLU (alpha=0.05))
model.add(keras.layers.Dense(fc6_dims))
model.add(keras.layers.LeakyReLU (alpha=0.05))
model.add(keras.layers.Dense(fc7_dims))
model.add(keras.layers.LeakyReLU (alpha=0.05))
model.add(keras.layers.Lambda(lambda x: K.tanh(0.1*x)))
model.add(keras.layers.Dense(2,activation=None))
model.compile(optimizer=Adam(learning_rate=lr), loss='mean_squared_error')
model = keras.models.load_model('strengthFlop.h5')

In [27]:
simHandsFlop = pd.read_csv('simHandsFlop.csv', index_col=0)
X = np.array(simHandsFlop[['val1', 'suit1', 'val2', 'suit2', 'f1val', 'f1suit', 'f2val','f2suit','f3val','f3suit']])
X_train = X[0:75000]
X_test = X[75000:]
y = np.array(simHandsFlop[['nwin', 'nloose']])
y_train = y[:75000]
y_test = y[75000:]

In [ ]:
model.fit(X_train, y_train, epochs=200, batch_size=512)

In [ ]:
model.evaluate(X_train, y_train)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
model.predict(X_train)

In [ ]:
model.predict(X_test)

In [ ]:
y_test

In [ ]:
model.predict(X_test)-y_test

In [ ]:
model.save('strengthFlop.h5')


In [24]:
modelFlop = load_model('strengthFlop.h5')

The same for turns

In [2]:
import pandas as pd
dfTurn = pd.read_csv('simHandsTurn.csv', index_col = False)
dfTurn

,val1,suit1,val2,suit2,f1val,f1suit,f2val,f2suit,f3val,f3suit,tval,tsuit,nwin,nloose,nsplit
0,5,1,13,3,13,0,10,1,10,3,8,1,0.786286,0.160857,0.052857
1,8,0,12,2,2,2,10,0,11,2,4,1,0.319429,0.659429,0.021143
2,8,2,13,0,9,2,12,0,10,1,11,0,0.799429,0.040000,0.160571
3,2,0,13,0,12,3,12,0,7,3,13,1,0.754857,0.162571,0.082571
4,5,0,10,3,8,0,11,1,14,0,10,2,0.532000,0.403429,0.064571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159995,2,3,4,2,12,1,13,3,10,0,7,1,0.048286,0.843714,0.108000
159996,7,1,11,0,12,0,10,0,4,2,10,3,0.291429,0.643143,0.065429
159997,13,1,14,2,12,1,10,3,12,2,7,0,0.602857,0.372000,0.025143
159998,3,3,6,0,9,3,9,0,2,0,5,3,0.180286,0.798000,0.021714


In [3]:
X_train = np.array(dfTurn.iloc[:150000,:12])
X_test = np.array(dfTurn.iloc[150000:,:12])
y_train = np.array(dfTurn.iloc[:150000, -3:-1])
y_test = np.array(dfTurn.iloc[150000:,-3:-1])

In [5]:
input_dims = 12
fc1_dims = 256
fc2_dims = 128
fc3_dims = 64
fc4_dims = 32
fc5_dims = 16
fc6_dims = 8
fc7_dims=4
lr = 0.001
modelTurn = keras.Sequential()
modelTurn.add(keras.layers.Dense(fc1_dims, input_dim=input_dims, activation = 'relu'))
#modelTurn.add(keras.layers.LeakyReLU (alpha=0.05))
modelTurn.add(keras.layers.Dense(fc2_dims, activation = 'relu'))
#modelTurn.add(keras.layers.LeakyReLU (alpha=0.05))
modelTurn.add(keras.layers.Dense(fc3_dims, activation = 'relu'))
#modelTurn.add(keras.layers.LeakyReLU (alpha=0.05))
modelTurn.add(keras.layers.Dense(fc4_dims, activation = 'relu'))
#modelTurn.add(keras.layers.LeakyReLU (alpha=0.05))
modelTurn.add(keras.layers.Dense(fc5_dims, activation = 'relu'))
#modelTurn.add(keras.layers.LeakyReLU (alpha=0.05))
modelTurn.add(keras.layers.Dense(fc6_dims, activation = 'relu'))
#modelTurn.add(keras.layers.LeakyReLU (alpha=0.05))
modelTurn.add(keras.layers.Dense(fc7_dims, activation = 'relu'))
#modelTurn.add(keras.layers.LeakyReLU (alpha=0.05))
#modelTurn.add(keras.layers.Dense(fc8_dims, activation = 'relu'))
#modelTurn.add(keras.layers.LeakyReLU (alpha=0.05))
modelTurn.add(keras.layers.Lambda(lambda x: K.tanh(0.1*x)))
modelTurn.add(keras.layers.Dense(2,activation=None))
modelTurn.compile(optimizer=Adam(learning_rate=lr), loss='mean_squared_error')

In [25]:
modelTurn = keras.models.load_model('strengthTurn.h5')
modelTurn.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_100 (Dense)            (None, 256)               3328      
_________________________________________________________________
dense_101 (Dense)            (None, 128)               32896     
_________________________________________________________________
dense_102 (Dense)            (None, 64)                8256      
_________________________________________________________________
dense_103 (Dense)            (None, 32)                2080      
_________________________________________________________________
dense_104 (Dense)            (None, 16)                528       
_________________________________________________________________
dense_105 (Dense)            (None, 8)                 136       
_________________________________________________________________
dense_106 (Dense)            (None, 4)               

In [ ]:
modelTurn.fit(X_train, y_train, epochs=100, batch_size=32)

In [ ]:
modelTurn.predict(X_test[:25])

In [ ]:
X_test[0]

In [ ]:
y_test[:25]

In [12]:
modelTurn.evaluate(X_train, y_train)

4688/4688 [==============================] - 4s 922us/step - loss: 0.0069


0.006862732581794262

In [14]:
modelTurn.evaluate(X_test, y_test)

313/313 [==============================] - 0s 914us/step - loss: 0.0084


0.008372490294277668

In [15]:
modelTurn.save('strengthTurn.h5')

Lastly let's build an evaluator for the river

In [2]:
import pandas as pd
dfRiver = pd.read_csv('simHandsRiver.csv', index_col = False)
dfRiver = dfRiver.append(pd.read_csv('simHandsRiverMore.csv', index_col = False))

In [3]:
dfRiver

,val1,suit1,val2,suit2,f1val,f1suit,f2val,f2suit,f3val,f3suit,tval,tsuit,rval,rsuit,nwin,nloose,nsplit
0,2,0,3,2,14,2,12,2,7,2,5,2,8,3,0.710101,0.289899,0.000000
1,5,3,8,2,11,3,3,0,9,0,3,2,3,1,0.000000,0.796970,0.203030
2,3,0,5,1,4,3,10,0,4,0,10,1,12,3,0.000000,0.593939,0.406061
3,4,0,13,3,6,2,5,0,7,0,11,0,14,1,0.260606,0.708081,0.031313
4,5,0,12,1,10,3,4,2,11,1,6,2,9,3,0.129293,0.837374,0.033333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1599995,4,3,14,3,6,0,5,1,3,2,7,3,11,3,0.853535,0.028283,0.118182
1599996,4,1,14,0,6,2,14,3,10,0,8,2,6,3,0.806061,0.147475,0.046465
1599997,9,2,9,3,7,0,3,0,4,1,8,1,14,0,0.703030,0.295960,0.001010
1599998,5,3,11,1,13,1,3,1,8,1,4,3,13,0,0.219192,0.771717,0.009091


In [4]:
X_train = np.array(dfRiver.iloc[:1800000,:14])
X_test = np.array(dfRiver.iloc[1800000:,:14])
y_train = np.array(dfRiver.iloc[:1800000, -3:-1])
y_test = np.array(dfRiver.iloc[1800000:,-3:-1])

In [13]:
input_dims = 14
fc1_dims = 256
fc2_dims = 128
fc3_dims = 64
fc4_dims = 32
fc5_dims = 16
fc6_dims = 18
fc7_dims = 4



lr = 0.0001
modelRiver = keras.Sequential()
modelRiver.add(keras.layers.Dense(fc1_dims, input_dim=input_dims, activation = 'relu'))
modelRiver.add(keras.layers.LeakyReLU (alpha=0.05))
modelRiver.add(keras.layers.Dense(fc2_dims, activation = 'relu'))
modelRiver.add(keras.layers.LeakyReLU (alpha=0.05))
modelRiver.add(keras.layers.Dense(fc3_dims, activation = 'relu'))
modelRiver.add(keras.layers.LeakyReLU (alpha=0.05))
modelRiver.add(keras.layers.Dense(fc4_dims, activation = 'relu'))
modelRiver.add(keras.layers.LeakyReLU (alpha=0.05))
modelRiver.add(keras.layers.Dense(fc5_dims, activation = 'relu'))
modelRiver.add(keras.layers.LeakyReLU (alpha=0.05))
modelRiver.add(keras.layers.Dense(fc6_dims, activation = 'relu'))
modelRiver.add(keras.layers.LeakyReLU (alpha=0.05))
modelRiver.add(keras.layers.Dense(fc7_dims, activation = 'relu'))
modelRiver.add(keras.layers.LeakyReLU (alpha=0.05))
#modelRiver.add(keras.layers.Dense(fc8_dims, activation = 'relu'))
#modelRiver.add(keras.layers.LeakyReLU (alpha=0.05))

modelRiver.add(keras.layers.Lambda(lambda x: K.tanh(0.1*x)))
modelRiver.add(keras.layers.Dense(2,activation=None))
modelRiver.compile(optimizer=Adam(learning_rate=lr), loss='mean_squared_error')

In [5]:
modelRiver = keras.models.load_model('strengthRiver.h5')
modelRiver.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_40 (Dense)             (None, 256)               3840      
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_41 (Dense)             (None, 128)               32896     
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_42 (Dense)             (None, 64)                8256      
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 64)                0         
_________________________________________________________________
dense_43 (Dense)             (None, 32)               

In [22]:
X_test[9]

array([ 5,  0, 13,  3, 11,  0,  6,  2, 12,  2,  6,  0, 12,  1],
      dtype=int64)

In [ ]:
#modelRiver = load_model('strengthRiver.h5')
for i in range(50):
    modelRiver.fit(X_train, y_train, epochs=20, batch_size=32)
    modelRiver.save('strengthRiver.h5')

In [15]:
modelRiver.predict(X_test[6:16])-y_test[6:16]

array([[ 0.01352796, -0.01435942],
       [-0.09991232,  0.0897559 ],
       [-0.02419723,  0.00802623],
       [ 0.00357963, -0.06325077],
       [ 0.00892413, -0.13723116],
       [ 0.04021648,  0.16860511],
       [-0.01161999,  0.02576104],
       [-0.00508   ,  0.0010395 ],
       [-0.01127839, -0.0015831 ],
       [-0.00579486,  0.01828718]])

In [13]:
np.set_printoptions(suppress=True)
y_predict = modelRiver.predict(X_test)-y_test
y_predict_large = y_predict[abs(y_predict)>0.1]

In [14]:
y_predict_large.sort()
y_predict_large[:100]

array([-0.98262086, -0.95790335, -0.94469953, -0.93719199, -0.93586783,
       -0.92690053, -0.92010016, -0.91987018, -0.91948981, -0.9192143 ,
       -0.90755154, -0.90206843, -0.89335627, -0.89328489, -0.89067104,
       -0.88925023, -0.88653174, -0.88542588, -0.884771  , -0.88085893,
       -0.877368  , -0.87713831, -0.87619864, -0.86925365, -0.86673589,
       -0.86275759, -0.86033178, -0.86010983, -0.85876344, -0.85758146,
       -0.85550286, -0.85440295, -0.84975376, -0.84842439, -0.84627254,
       -0.84226031, -0.84099094, -0.83752181, -0.83502094, -0.83372953,
       -0.83059712, -0.82702353, -0.82041313, -0.81758027, -0.81419946,
       -0.81057886, -0.81039038, -0.80884578, -0.80797716, -0.8055819 ,
       -0.80523194, -0.80467442, -0.80393179, -0.8032746 , -0.80226048,
       -0.80173189, -0.800919  , -0.79869545, -0.7984505 , -0.79172484,
       -0.79159806, -0.78914575, -0.78755524, -0.78741525, -0.7871072 ,
       -0.78417244, -0.77822606, -0.77771349, -0.77682344, -0.77

In [19]:
y_test[6:16]

array([[0.9       , 0.09090909],
       [0.14545455, 0.80909091],
       [0.12525253, 0.86565657],
       [0.44545455, 0.46060606],
       [0.        , 0.96666667],
       [0.43232323, 0.56161616],
       [0.91212121, 0.08383838],
       [0.21212121, 0.77878788],
       [0.94646465, 0.05050505],
       [0.86969697, 0.01212121]])

In [ ]:
np.set_printoptions(suppress=True)
modelRiver.predict(X_test[25:50])-y_test[25:50]

In [116]:
modelRiver.predict(np.array([X_test[0]]))[0][1]

0.55815035

In [ ]:
modelRiver.evaluate(X_train, y_train)

In [6]:
modelRiver.evaluate(X_test, y_test)

3750/3750 [==============================] - 4s 895us/step - loss: 0.0063


0.006272866856306791

In [34]:
modelRiver.save('strengthRiver.h5')

Besides the hand strength of the player, we want the average expected hand strength on that board and a variance meassure. Preflop it is trivial, since it is simply the avarage likelihood of winning, and lossing, and the variance.

In [21]:
import pandas as pd
simHandsPreflop = pd.read_csv('simHandsPreflop.csv', index_col=False)
simHandsPreflop['nsplit'].std()

0.01437192120950129

In [23]:
strengthPreflop = pd.read_csv('strengthPreflop.csv', index_col= False)
strengthPreflop['nwinVil'] = simHandsPreflop['nwin'].mean()
strengthPreflop['nlooseVil'] = simHandsPreflop['nloose'].mean()
strengthPreflop['nsplitVil'] = simHandsPreflop['nsplit'].mean()
strengthPreflop['stdWinVil'] = simHandsPreflop['nwin'].std()
strengthPreflop['stdLooseVil'] = simHandsPreflop['nloose'].std()
strengthPreflop['stdSplitVil'] = simHandsPreflop['nsplit'].std()
strengthPreflop.to_csv('strengthPreflop.csv', index = False)

Let's train neural networks to also predict the avg handstrength on each board as well as variance meassures for those hand strenghth.

In [50]:
import numpy as np
import pandas as pd
nwinVil = np.load('Npfiles/nwinVil'+str(0) +'.npy')
nlooseVil = np.load('Npfiles/nlooseVil'+str(0) +'.npy')
stdWinVil = np.load('Npfiles/stdWinVil'+str(0) +'.npy')
stdLooseVil = np.load('Npfiles/stdLooseVil'+str(0) +'.npy')
arr = np.array([nwinVil, nlooseVil, stdWinVil, stdLooseVil])

for i in range(1,8):
    nwinVil = np.append(nwinVil,np.load('Npfiles/nwinVil'+str(i) +'.npy'))
    nlooseVil = np.append(nlooseVil, np.load('Npfiles/nlooseVil'+str(i) +'.npy'))
    stdWinVil = np.append(stdWinVil,np.load('Npfiles/stdWinVil'+str(i) +'.npy'))
    stdLooseVil = np.append(stdLooseVil,np.load('Npfiles/stdLooseVil'+str(i) +'.npy'))
    
dfFlopMeassures=pd.DataFrame(np.array([nwinVil, nlooseVil, stdWinVil, stdLooseVil]).transpose())
dfFlopMeassures.columns = (['nwinVil', 'nlooseVil', 'stdWinVil', 'stdLooseVil'])
dfFlopMeassures.head()


,nwinVil,nlooseVil,stdWinVil,stdLooseVil
0,0.606727,0.431889,0.209281,0.209281
1,0.590272,0.466472,0.231694,0.231694
2,0.553225,0.492703,0.234816,0.234816
3,0.600785,0.432018,0.221529,0.221529
4,0.589939,0.451438,0.217125,0.217125


We see that stdLoose and stdWin are the same. This should not be. The error was made in the generation of these data. So we will only use stdWin as variance meassure. Training the data would take too long and the information increase is minimal.

Let's first combine those meassures with the ones we allready have.

In [ ]:
simHandsFlop[['nwinVil', 'nlooseVil', 'stdWinVil']] = dfFlopMeassures[['nwinVil', 'nlooseVil', 'stdWinVil']]
simHandsFlop.drop(columns = ['tval','tsuit', 'rval', 'rsuit'], inplace=True)

In [62]:
simHandsFlop

,val1,suit1,val2,suit2,f1val,f1suit,f2val,f2suit,f3val,f3suit,nwin,nloose,nsplit,nwinVil,nlooseVil,stdWinVil
0,4,2,8,2,2,3,9,3,10,1,0.2120,0.7422,0.0458,0.606727,0.431889,0.209281
1,5,2,10,3,8,3,11,2,13,0,0.3080,0.6248,0.0672,0.590272,0.466472,0.231694
2,12,1,13,3,10,0,10,2,12,2,0.7650,0.2038,0.0312,0.553225,0.492703,0.234816
3,7,0,10,3,2,1,11,0,13,1,0.3252,0.6280,0.0468,0.600785,0.432018,0.221529
4,10,2,14,3,2,2,7,1,13,2,0.5306,0.4516,0.0178,0.589939,0.451438,0.217125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,11,3,12,1,5,1,5,2,9,3,0.4462,0.5020,0.0518,0.574971,0.464509,0.217393
79996,6,2,7,2,2,3,10,3,12,3,0.1470,0.7866,0.0664,0.586323,0.437367,0.210976
79997,12,0,14,1,6,2,8,2,10,0,0.4640,0.5140,0.0220,0.583135,0.472573,0.197530
79998,8,0,13,1,4,2,9,3,10,3,0.3910,0.5816,0.0274,0.609806,0.445365,0.207131


In [63]:
simHandsFlop.to_csv('simHandsFlopVil.csv', index=False)

In [1]:
simHandsFlopVil = pd.read_csv('simHandsFlopVil.csv',index_col=False)
simHandsFlopVil.head()

NameError: name 'pd' is not defined

In [70]:
simHandsFlopVil.drop(columns='nsplit', inplace=True)

Let's train a neural network to predict those values. We take the same as before, just with 5 instead of two outputs.

In [71]:
input_dims = 10
fc1_dims = 256
fc2_dims = 128
fc3_dims = 64
fc4_dims = 32
fc5_dims = 16
fc6_dims = 8
fc7_dims = 4
lr = 0.001
modelFlop = keras.Sequential()
modelFlop.add(keras.layers.Dense(fc1_dims, input_dim=input_dims))
modelFlop.add(keras.layers.LeakyReLU (alpha=0.05))
modelFlop.add(keras.layers.Dense(fc2_dims))
modelFlop.add(keras.layers.LeakyReLU (alpha=0.05))
modelFlop.add(keras.layers.Dense(fc3_dims))
modelFlop.add(keras.layers.LeakyReLU (alpha=0.05))
modelFlop.add(keras.layers.Dense(fc4_dims))
modelFlop.add(keras.layers.LeakyReLU (alpha=0.05))
modelFlop.add(keras.layers.Dense(fc5_dims))
modelFlop.add(keras.layers.LeakyReLU (alpha=0.05))
modelFlop.add(keras.layers.Dense(fc6_dims))
modelFlop.add(keras.layers.LeakyReLU (alpha=0.05))
modelFlop.add(keras.layers.Dense(fc7_dims))
modelFlop.add(keras.layers.LeakyReLU (alpha=0.05))
modelFlop.add(keras.layers.Lambda(lambda x: K.tanh(0.1*x)))
modelFlop.add(keras.layers.Dense(5,activation=None))
modelFlop.compile(optimizer=Adam(learning_rate=lr), loss='mean_squared_error')

In [72]:
X = np.array(simHandsFlopVil[['val1', 'suit1', 'val2', 'suit2', 'f1val', 'f1suit', 'f2val','f2suit','f3val','f3suit']])
X_train = X[0:75000]
X_test = X[75000:]
y = np.array(simHandsFlopVil[['nwin', 'nloose','nwinVil', 'nlooseVil','stdWinVil']])
y_train = y[:75000]
y_test = y[75000:]

In [ ]:
modelFlop.fit(X_train, y_train, epochs=100, batch_size=32)

In [77]:
modelFlop.evaluate(X_train, y_train)

2344/2344 [==============================] - 6s 3ms/step - loss: 0.0014


0.0013954867608845234

In [81]:
modelFlop.evaluate(X_test, y_test)

157/157 [==============================] - 0s 3ms/step - loss: 1.7841e-04


0.00017841090448200703

In [ ]:
model.predict(X_train)

In [ ]:
model.predict(X_test)

In [ ]:
y_test

In [82]:
modelFlop.predict(X_test)-y_test

array([[-0.00022846,  0.0106678 ,  0.00906563, -0.00060106,  0.00092159],
       [ 0.03085328, -0.02643636, -0.00020178, -0.0025875 , -0.00967476],
       [ 0.02655813, -0.03629981, -0.00783804,  0.00549031,  0.00443127],
       ...,
       [-0.00333877,  0.00481635,  0.00380194, -0.00169691,  0.00182256],
       [-0.00656503,  0.00645597, -0.00752276, -0.00065157, -0.00508955],
       [ 0.01554059, -0.00179418, -0.00547619, -0.00434384,  0.00490126]])

In [83]:
modelFlop.save('strengthFlop2.h5')


Let's do it for Turn also

In [2]:
import numpy as np
import pandas as pd
nwinVil = np.load('Npfiles/tnwinVil'+str(0) +'.npy')
nlooseVil = np.load('Npfiles/tnlooseVil'+str(0) +'.npy')
stdWinVil = np.load('Npfiles/tstdWinVil'+str(0) +'.npy')
stdLooseVil = np.load('Npfiles/tstdLooseVil'+str(0) +'.npy')
arr = np.array([nwinVil, nlooseVil, stdWinVil, stdLooseVil])

for i in range(1,8):
    nwinVil = np.append(nwinVil,np.load('Npfiles/tnwinVil'+str(i) +'.npy'))
    nlooseVil = np.append(nlooseVil, np.load('Npfiles/tnlooseVil'+str(i) +'.npy'))
    stdWinVil = np.append(stdWinVil,np.load('Npfiles/tstdWinVil'+str(i) +'.npy'))
    stdLooseVil = np.append(stdLooseVil,np.load('Npfiles/tstdLooseVil'+str(i) +'.npy'))
    
dfTurnMeassures=pd.DataFrame(np.array([nwinVil, nlooseVil, stdWinVil, stdLooseVil]).transpose())
dfTurnMeassures.columns = (['nwinVil', 'nlooseVil', 'stdWinVil', 'stdLooseVil'])
dfTurnMeassures.head()


,nwinVil,nlooseVil,stdWinVil,stdLooseVil
0,0.509220,0.428819,0.263491,0.263491
1,0.659640,0.331385,0.267622,0.267622
2,0.510218,0.431432,0.253778,0.253778
3,0.504427,0.455482,0.230119,0.230119
4,0.552759,0.408489,0.267769,0.267769


We see that stdLoose and stdWin are the same. This should not be. The error was made in the generation of these data. So we will only use stdWin as variance meassure. Training the data would take too long and the information increase is minimal.

Let's first combine those meassures with the ones we allready have.

In [3]:
simHandsTurn = pd.read_csv('simHandsTurn.csv')
simHandsTurn[['nwinVil', 'nlooseVil', 'stdWinVil']] = dfTurnMeassures[['nwinVil', 'nlooseVil', 'stdWinVil']]
simHandsTurn.head()

,val1,suit1,val2,suit2,f1val,f1suit,f2val,f2suit,f3val,f3suit,tval,tsuit,nwin,nloose,nsplit,nwinVil,nlooseVil,stdWinVil
0,5,1,13,3,13,0,10,1,10,3,8,1,0.786286,0.160857,0.052857,0.509220,0.428819,0.263491
1,8,0,12,2,2,2,10,0,11,2,4,1,0.319429,0.659429,0.021143,0.659640,0.331385,0.267622
2,8,2,13,0,9,2,12,0,10,1,11,0,0.799429,0.040000,0.160571,0.510218,0.431432,0.253778
3,2,0,13,0,12,3,12,0,7,3,13,1,0.754857,0.162571,0.082571,0.504427,0.455482,0.230119
4,5,0,10,3,8,0,11,1,14,0,10,2,0.532000,0.403429,0.064571,0.552759,0.408489,0.267769


In [88]:
simHandsTurn.to_csv('simHandsTurnVil.csv', index=False)

In [2]:
import pandas as pd

simHandsTurnVil = pd.read_csv('simHandsTurnVil.csv',index_col=False)
simHandsTurnVil.head()

,val1,suit1,val2,suit2,f1val,f1suit,f2val,f2suit,f3val,f3suit,tval,tsuit,nwin,nloose,nsplit,nwinVil,nlooseVil,stdWinVil
0,5,1,13,3,13,0,10,1,10,3,8,1,0.786286,0.160857,0.052857,0.509220,0.428819,0.263491
1,8,0,12,2,2,2,10,0,11,2,4,1,0.319429,0.659429,0.021143,0.659640,0.331385,0.267622
2,8,2,13,0,9,2,12,0,10,1,11,0,0.799429,0.040000,0.160571,0.510218,0.431432,0.253778
3,2,0,13,0,12,3,12,0,7,3,13,1,0.754857,0.162571,0.082571,0.504428,0.455482,0.230119
4,5,0,10,3,8,0,11,1,14,0,10,2,0.532000,0.403429,0.064571,0.552759,0.408489,0.267769


In [3]:
simHandsTurnVil.drop(columns='nsplit', inplace=True)

Let's train a neural network to predict those values. We take the same as before, just with 5 instead of two outputs.

In [5]:
input_dims = 12
fc1_dims = 256
fc2_dims = 128
fc3_dims = 64
fc4_dims = 32
fc5_dims = 16
fc6_dims = 8
fc7_dims = 4
lr = 0.0001
modelTurnVil = keras.Sequential()
modelTurnVil.add(keras.layers.Dense(fc1_dims, input_dim=input_dims))
modelTurnVil.add(keras.layers.LeakyReLU (alpha=0.05))
modelTurnVil.add(keras.layers.Dense(fc2_dims))
modelTurnVil.add(keras.layers.LeakyReLU (alpha=0.05))
modelTurnVil.add(keras.layers.Dense(fc3_dims))
modelTurnVil.add(keras.layers.LeakyReLU (alpha=0.05))
modelTurnVil.add(keras.layers.Dense(fc4_dims))
modelTurnVil.add(keras.layers.LeakyReLU (alpha=0.05))
modelTurnVil.add(keras.layers.Dense(fc5_dims))
modelTurnVil.add(keras.layers.LeakyReLU (alpha=0.05))
modelTurnVil.add(keras.layers.Dense(fc6_dims))
modelTurnVil.add(keras.layers.LeakyReLU (alpha=0.05))
modelTurnVil.add(keras.layers.Dense(fc7_dims))
modelTurnVil.add(keras.layers.LeakyReLU (alpha=0.05))
modelTurnVil.add(keras.layers.Lambda(lambda x: K.sigmoid(0.1*x)))
modelTurnVil.add(keras.layers.Dense(5,activation=None))
modelTurnVil.compile(optimizer=Adam(learning_rate=lr), loss='mean_squared_error')

In [10]:
simHandsTurnVil.shape

(160000, 17)

In [10]:
X = np.array(simHandsTurnVil[['val1', 'suit1', 'val2', 'suit2', 'f1val', 'f1suit', 'f2val','f2suit','f3val','f3suit', 'tval', 'tsuit']])
X_train = X[0:150000]
X_test = X[150000:]
y = np.array(simHandsTurnVil[['nwin', 'nloose','nwinVil', 'nlooseVil','stdWinVil']])
y_train = y[:150000]
y_test = y[150000:]

In [11]:
modelTurnVil.fit(X_train, y_train, epochs=1, batch_size=16)

2021-12-31 19:12:29.185789: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 14400000 exceeds 10% of free system memory.


9375/9375 [==============================] - 27s 3ms/step - loss: 0.0025


In [100]:
modelTurnVil.evaluate(X_train, y_train)

4688/4688 [==============================] - 12s 3ms/step - loss: 0.0024


0.0023814637679606676

In [24]:
modelTurnVil.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.0030


0.0030039502307772636

In [9]:
modelTurnVil.predict(X_test)[modelTurnVil.predict(X_test)>1]

array([], dtype=float32)

In [35]:
modelTurnVil.save('strengthTurn2.h5')


In [4]:
modelTurnVil = load_model('strengthTurn2.h5')
modelTurnVil.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 256)               3328      
                                                                 
 leaky_re_lu_7 (LeakyReLU)   (None, 256)               0         
                                                                 
 dense_9 (Dense)             (None, 128)               32896     
                                                                 
 leaky_re_lu_8 (LeakyReLU)   (None, 128)               0         
                                                                 
 dense_10 (Dense)            (None, 64)                8256      
                                                                 
 leaky_re_lu_9 (LeakyReLU)   (None, 64)                0         
                                                                 
 dense_11 (Dense)            (None, 32)               

2021-12-31 19:11:37.323872: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-31 19:11:37.323905: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-31 19:11:37.323927: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cromi-Lenovo-V15-ADA): /proc/driver/nvidia/version does not exist
2021-12-31 19:11:37.324170: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


And finally for the river

In [6]:
import numpy as np
import pandas as pd
nwinVil = np.load('Npfiles/rnwinVil'+str(0) +'.npy')
nlooseVil = np.load('Npfiles/rnlooseVil'+str(0) +'.npy')
stdWinVil = np.load('Npfiles/rstdWinVil'+str(0) +'.npy')
stdLooseVil = np.load('Npfiles/rstdLooseVil'+str(0) +'.npy')
arr = np.array([nwinVil, nlooseVil, stdWinVil, stdLooseVil])

for i in range(1,8):
    nwinVil = np.append(nwinVil,np.load('Npfiles/rnwinVil'+str(i) +'.npy'))
    nlooseVil = np.append(nlooseVil, np.load('Npfiles/rnlooseVil'+str(i) +'.npy'))
    stdWinVil = np.append(stdWinVil,np.load('Npfiles/rstdWinVil'+str(i) +'.npy'))
    stdLooseVil = np.append(stdLooseVil,np.load('Npfiles/rstdLooseVil'+str(i) +'.npy'))
    
dfRiverMeassures=pd.DataFrame(np.array([nwinVil, nlooseVil, stdWinVil, stdLooseVil]).transpose())
dfRiverMeassures.columns = (['nwinVil', 'nlooseVil', 'stdWinVil', 'stdLooseVil'])
dfRiverMeassures.head()


,nwinVil,nlooseVil,stdWinVil,stdLooseVil
0,0.721921,0.263837,0.306961,0.306961
1,0.697093,0.289365,0.312681,0.312681
2,0.712263,0.239752,0.343167,0.343167
3,0.697303,0.286709,0.337385,0.337385
4,0.670045,0.308301,0.326259,0.326259


We see that stdLoose and stdWin are the same. This should not be. The error was made in the generation of these data. So we will only use stdWin as variance meassure. Training the data would take too long and the information increase is minimal.

Let's first combine those meassures with the ones we allready have.

In [7]:
simHandsRiver = pd.read_csv('simHandsRiverMore.csv')
simHandsRiver[['nwinVil', 'nlooseVil', 'stdWinVil']] = dfRiverMeassures[['nwinVil', 'nlooseVil', 'stdWinVil']]
simHandsRiver.head()

NameError: name 'dfRiverMeassures' is not defined

In [4]:
simHandsRiver.to_csv('simHandsRiverVil.csv', index=False)

In [5]:
import pandas as pd
simHandsRiverVil = pd.read_csv('simHandsRiverVil.csv',index_col=False)
simHandsRiverVil.head()

,val1,suit1,val2,suit2,f1val,f1suit,f2val,f2suit,f3val,f3suit,tval,tsuit,rval,rsuit,nwin,nloose,nsplit,nwinVil,nlooseVil,stdWinVil
0,5,2,11,0,4,1,7,0,13,1,11,2,9,0,0.707071,0.262626,0.030303,0.721921,0.263837,0.306961
1,3,0,14,1,8,2,12,1,5,0,7,2,10,2,0.248485,0.710101,0.041414,0.697093,0.289365,0.312681
2,3,0,3,2,10,1,5,3,12,2,5,2,13,0,0.490909,0.508081,0.001010,0.712263,0.239752,0.343167
3,6,0,10,3,13,1,11,2,2,0,13,0,6,2,0.673737,0.265657,0.060606,0.697303,0.286709,0.337385
4,3,1,13,2,12,2,8,0,7,2,11,2,14,0,0.278788,0.663636,0.057576,0.670045,0.308301,0.326259


In [6]:
simHandsRiverVil.drop(columns='nsplit', inplace=True)

Let's train a neural network to predict those values. We take the same as before, just with 5 instead of two outputs.

In [17]:
input_dims = 14
fc1_dims = 256
fc2_dims = 128
fc3_dims = 64
fc4_dims = 32
fc5_dims = 16
fc6_dims = 8
fc7_dims = 4
lr = 0.0001
modelRiverVil = keras.Sequential()
modelRiverVil.add(keras.layers.Dense(fc1_dims, input_dim=input_dims))
modelRiverVil.add(keras.layers.LeakyReLU (alpha=0.05))
modelRiverVil.add(keras.layers.Dense(fc2_dims))
modelRiverVil.add(keras.layers.LeakyReLU (alpha=0.05))
modelRiverVil.add(keras.layers.Dense(fc3_dims))
modelRiverVil.add(keras.layers.LeakyReLU (alpha=0.05))
modelRiverVil.add(keras.layers.Dense(fc4_dims))
modelRiverVil.add(keras.layers.LeakyReLU (alpha=0.05))
modelRiverVil.add(keras.layers.Dense(fc5_dims))
modelRiverVil.add(keras.layers.LeakyReLU (alpha=0.05))
modelRiverVil.add(keras.layers.Dense(fc6_dims))
modelRiverVil.add(keras.layers.LeakyReLU (alpha=0.05))
modelRiverVil.add(keras.layers.Dense(fc7_dims))
modelRiverVil.add(keras.layers.LeakyReLU (alpha=0.05))
modelRiverVil.add(keras.layers.Dense(5,activation=None))
modelRiverVil.add(keras.layers.Lambda(lambda x: K.sigmoid(0.1*x)))

modelRiverVil.compile(optimizer=Adam(learning_rate=lr), loss='mean_squared_error')

In [5]:
simHandsRiverVil.shape

(1600000, 19)

In [8]:
import numpy as np
X = np.array(simHandsRiverVil[['val1', 'suit1', 'val2', 'suit2', 'f1val', 'f1suit', 'f2val','f2suit','f3val','f3suit', 'tval', 'tsuit', 'rval', 'rsuit']])
X_train = X[0:1555000]
X_test = X[1555000:]
y = np.array(simHandsRiverVil[['nwin', 'nloose','nwinVil', 'nlooseVil','stdWinVil']])
y_train = y[:1555000]
y_test = y[1555000:]

In [13]:
modelRiverVil = load_model('strengthRiver2New.h5')

2022-01-01 11:50:25.527711: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-01 11:50:25.527754: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-01 11:50:25.527789: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cromi-Lenovo-V15-ADA): /proc/driver/nvidia/version does not exist
2022-01-01 11:50:25.528221: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [16]:
modelRiverVil.fit(X_train, y_train, epochs=5, batch_size=16)

Epoch 1/5
97188/97188 [==============================] - 281s 3ms/step - loss: 0.0096
Epoch 2/5
97188/97188 [==============================] - 290s 3ms/step - loss: 0.0094
Epoch 3/5
97188/97188 [==============================] - 282s 3ms/step - loss: 0.0093
Epoch 4/5
97188/97188 [==============================] - 282s 3ms/step - loss: 0.0091
Epoch 5/5
97188/97188 [==============================] - 280s 3ms/step - loss: 0.0089


In [ ]:
modelRiverVil.evaluate(X_train, y_train)

In [14]:
modelRiverVil.evaluate(X_test, y_test)

1407/1407 [==============================] - 3s 2ms/step - loss: 0.0098


0.00977681390941143

In [16]:
modelRiverVil.predict(X_test)[modelRiverVil.predict(X_test)>1]

array([1.0170047, 1.0020736, 1.0615225, 1.0045848, 1.0059962, 1.0237631,
       1.0247722, 1.004365 , 1.0045214, 1.0219518, 1.0125753],
      dtype=float32)

In [4]:
modelRiverVil.save('strengthRiver2New.h5')


NameError: name 'modelRiverVil' is not defined